# To load data from NAS - python sessdfGenerator.py in conda env 

In [2]:
from utils.opconNosepokeFunctions import *
from utils.supplementaryFunctions import *
from scipy.optimize import minimize
from scipy.stats import entropy
from scipy.stats import ttest_rel
%load_ext autoreload
%autoreload 2
import pandas as pd
import numpy as np

from utils import plotSettings

# import pickle 
# with open('L:/4portProb_processed/cleandf.pkl', 'rb') as f:
#     df = pickle.load(f)

sessdf = pd.read_csv('S:/fileTransferFromWindows/sessdf.csv')

window = 5
# sessdf = df
sessdf['rr'] = sessdf.groupby(['animal', 'session'], as_index = False).reward.rolling(window, center=True).mean().reward
bin_size = 50
sessdf['sess_bin'] = sessdf.groupby(['animal', 'task'])['session'].transform(lambda x: pd.cut(x, bins=range(0, x.max() + bin_size, bin_size), labels=False, right=False)+1)

<Figure size 640x480 with 0 Axes>

In [3]:
an_dict = {
    'unstr'  : ['test05022023', 'Blissey', 'Chikorita', 'Darkrai', 'Eevee', 
                'Goldeen', 'Hoppip', 'Inkay', 'Jirachi', 'Kirlia', 'Mesprit',
                'Nidorina', 'Oddish', 'Phione', 'Quilava', 'Raltz', 'Shinx',
                'Togepi', 'Umbreon', 'Vulpix', 'Xatu', 'Yanma', 'Zacian',
                'Alakazam', 'Bayleef', 'Cresselia', 'Emolga', 'Giratina',
                'Haxorus', 'Ivysaur', 'Jigglypuff', 'Lugia', 'Ninetales', 'Onix',
                'Pichu', 'Quaxly', 'Sableye', 'Torchic', 'Uxie', 'Vanillish',
                'Whismur', 'Xerneas', 'Yamper', 'Zorua'],
    'dls'    : ['Blissey', 'Darkrai', 'Inkay' , 'Mesprit', 'Raltz', 'Zacian', 'Alakazam', 'Quilava', 'Torchic', 'Uxie', 'Vanillish', 'Yamper', 'Zorua'],
    'dms'    : ['Hoppip', 'Kirlia', 'Nidorina', 'Togepi', 'Xatu', 'Yanma', 'Giratina', 'Ivysaur', 'Lugia', 'Whismur', 'Quaxly', 'Sableye'],
    'sham'   : ['test05022023', 'Jirachi', 'Goldeen', 'Phione', 'Umbreon', 'Vulpix', 'Emolga', 'Haxorus', 'Jigglypuff'],
    'ds'     : ['Chikorita', 'Eevee'],
    'str'    : ['test05022023', 'Blissey', 'Chikorita', 'Darkrai', 'Eevee'],
    'oe_drive' : ['Shinx', 'Dratini', 'Bayleef', 'Onix', 'Pichu', 'Xerneas']
}

In [4]:
# Remove sessions (per animal) that have any duplicated trials (rows)
# Assumes 'animal' and 'session' columns exist in sessdf

def remove_sessions_with_duplicate_trials(df, animal_col='animal', session_col='session'):
    # Find sessions (per animal) with any duplicated trials based on 'trialstart' and 'eptime'
    trial_cols = ['trialstart', 'eptime']
    to_remove = set()
    for (animal, session), group in df.groupby([animal_col, session_col]):
        # If any duplicated trial in this group, mark for removal
        if group.duplicated(subset=trial_cols, keep=False).any():
            # print(f"Session {animal}, {session} has duplicated trials based on {trial_cols}")
            to_remove.add((animal, session))
    # Build mask for rows to keep
    mask = ~(df[[animal_col, session_col]].apply(tuple, axis=1).isin(to_remove))
    return df[mask].copy()

# Apply to sessdf
sessdf = remove_sessions_with_duplicate_trials(sessdf)


In [5]:
# sessdf.drop(columns = 'Unnamed: 0', inplace = True)
exclude = ['[ 20  20  20 100]', '[0 0 0 0]', '[0]', '[0 0]',
    '[1000   80]', '[30]', '[40]', '[70]']
sessdf = sessdf[~sessdf.rewprobfull.isin(exclude)]
sessdf = sessdf[~(sessdf.animal == 'Raikou')]

In [6]:
from utils.dfLoading import add_block_groups
sessdf = add_block_groups(sessdf)
sessdf.columns

Index(['Unnamed: 0', 'trial', 'trialstart', 'port', 'reward', 'trialend',
       'session', 'rewprob', 'eptime', 'task', 'rewprobfull', 'animal',
       'datetime', 'rr', 'sess_bin', 'sess_block', 'block_group'],
      dtype='object')

In [45]:
# save all data as .mat files for every animal
# first folder - all data
# from utils.dfLoading import add_block_groups
# blocked_df = add_block_groups(sessdf)
# blocked_df = blocked_df.rename(columns = {'session':'block', 'sess_block': 'session'})#.drop(columns = "Unnamed: 0")
# blocked_df['datetime'] = pd.to_datetime(blocked_df['datetime'])

# ## week relative to each animal-task first date
# blocked_df['week_task'] = blocked_df.groupby(['animal', 'task'])['datetime'].transform(
#     lambda x: ((x - x.min()).dt.days // 7) + 1
# )

# select week 1 rows per animal-task   #(blocked_df['week_task'] == 8)& 
week1_per_animal_task = blocked_df[(blocked_df.task == 'sham')]
# from scipy.io import savemat
# savemat("week1_data_RS_20251118_arr.mat", week1_per_animal_task[['animal', 'session','block','block_group', 'port', 'reward', 'week_task']].to_numpy(), appendmat=False)

# group or iterate as needed:
# for (animal, task), group in week1_per_animal_task.groupby(['animal', 'task']):
#     print(animal, task, group.shape)

In [27]:
ratdf = sessdf[sessdf.animal == 'Jirachi']
from utils.dfLoading import add_block_groups

ratdf = add_block_groups(ratdf)

c:\Users\dlab\rishika_sim\utils\dfLoading.py:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['sess_block'] = 0
c:\Users\dlab\rishika_sim\utils\dfLoading.py:63: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['sess_block'] = df.groupby(['animal', 'task']).sess_block.cumsum()
c:\Users\dlab\rishika_sim\utils\dfLoading.py:65: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentat

In [46]:
l = []
week1_choices = {}
week1_rewards = {}
week1_block_num = {}
week1_rewprob = {}
week1 = []
for (animal, group) in week1_per_animal_task[['animal', 'session', 'port', 'reward', 'week_task', 'block_group', 'rewprobfull']].groupby('animal'):
    l.append([group.session.nunique(), group.session.value_counts().max()])
    choices = np.ones(shape = l[-1])*np.nan
    rewards = np.ones(shape = l[-1])*np.nan
    block_num = np.ones(shape = l[-1])*np.nan
    rewprob = np.ones(shape = (group.session.nunique(), group.session.value_counts().max(), 4))*np.nan

    for idx, (sess, sess_group) in enumerate(group.groupby('session')):
        if (animal == 'Umbreon') & (sess != 362):
            l_sess = sess_group.port.to_numpy().shape[0]
            choices[idx, :l_sess] = sess_group.port.to_numpy()
            rewards[idx, :l_sess] = sess_group.reward.to_numpy()
            block_num[idx, :l_sess] = sess_group.block_group.to_numpy()
            rewprob[idx, :l_sess, :] = np.array([i.strip('[]').split(' ') for i in sess_group['rewprobfull']], dtype = int)
    week1_choices[animal] = choices
    week1_rewards[animal] = rewards
    week1_block_num[animal] = block_num
    week1_rewprob[animal] = rewprob
    print(animal)

    week1.append({'choices':week1_choices[animal], 'rewards':week1_rewards[animal], 'block_num':week1_block_num[animal], 'rewprob':week1_rewprob[animal], 'animal':animal})


Emolga
Haxorus
Jigglypuff
Jirachi
Phione
Umbreon
Vulpix
test05022023


In [47]:
from scipy.io import savemat
savemat("data/sham_data_20251118_RS.mat", {'sham':week1}, appendmat=True)

# Initial bias

In [94]:
row_num = 100
cond = 'dls'
nth_port = (sessdf[(sessdf.task == 'unstr') & (sessdf.block_group == 1) & (sessdf.sess_bin>=4)]
                               .groupby(['animal', 'session'])
                               .filter(lambda x: x.reward.size >= 100)
                               .groupby(['animal', 'session'])
                               .nth(row_num))
# sns.displot(data = first_10_ports, x = 'port', col = 'animal', discrete = True,
#             common_norm = False, col_wrap = 7, stat = 'probability', color = 'xkcd:cornflower')
init_port_prop = nth_port.groupby('animal').port.value_counts(normalize = True, sort = True).reset_index()
init_port_prop['highness'] = init_port_prop.groupby('animal').cumcount()+1
init_port_prop = init_port_prop.drop(columns = 'port')
pref_mat = init_port_prop.set_index(['animal', 'highness']).unstack()

In [95]:
plt.figure()
ax = plt.subplot(121)
ax.bar(np.arange(1,5),nth_port.port.value_counts(normalize=True), color = 'xkcd:cornflower')
ax.set_xticks(np.arange(1,5), np.arange(1,5), color = 'grey', fontsize = 'large')
ax.set_xlabel('Average choice', fontsize = 'x-large')
# ax.set_yticks(color = 'grey', fontsize = 'large')
ax.set_ylabel(f'Probability on trial {row_num}', fontsize = 'x-large')
sns.despine()
plt.tight_layout()
ax = plt.subplot(122)
ax.bar(np.arange(1,5),np.mean(pref_mat, axis = 0), color = 'xkcd:cornflower')
ax.set_xticks(np.arange(1,5), ['High', '', '', 'Low'],color = 'grey', fontsize = 'large')
ax.set_xlabel('Preferred choice', fontsize = 'x-large')
# ax.set_yticks(color = 'grey', fontsize = 'large')
ax.set_ylabel(f'Probability on trial {row_num}', fontsize = 'x-large')
sns.despine()
plt.tight_layout()

In [ ]:
# comparison pairs
cond_l = ["dls", "dms", 'sham']
for cond in cond_l:
    # calc_nth port
    print(cond)

# number of trials

In [79]:
# groupby cond and animal
from utils.plotSettings import *
countdf = sessdf.groupby(['animal', 'task'],
                         as_index = False)['sess_block'].value_counts().rename(columns = {'count':'sesscount'})

cond = 'sham'
an_list = sessdf[sessdf.task == cond].animal.sort_values().unique()

he = countdf[countdf.task.isin([cond, 'unstr'])& 
             (countdf.animal.isin(an_list))].groupby(['animal', 'task'], as_index = False).sesscount.median()

y = np.vstack([he[he.task == 'unstr'].sesscount.values, he[he.task == cond].sesscount.values]).T

# plot now!
fig = plt.figure(figsize = (3.5, 4.5))
ax = plt.subplot(111)
ax.bar([0, 1], [np.mean(y[:, 0]), np.mean(y[:, 1])], width = 0.4, color = ['xkcd:cornflower', 'xkcd:slate grey'], alpha = 0.5)
ax.plot([0]*len(y[:, 0]), y[:, 0], 'o', color = 'xkcd:cornflower', alpha = 0.7)
ax.plot([1]*len(y[:, 1]), y[:, 1], 'o', color = 'xkcd:slate grey', alpha = 0.7)

# for lines between points
for i in range(y.shape[0]):
    ax.plot(y[i, :], color = 'xkcd:slate grey', linewidth = 0.7, alpha = 0.3)

# sns.pointplot(data = countdf[countdf.task.isin([cond, 'unstr'])& (countdf.animal.isin(an_list))],
#                 order = ['unstr', cond], hue_order =['unstr', cond], 
#               y = 'sesscount', x = 'task', hue = 'task', errorbar = None, estimator = 'mean', 
#             palette = ['xkcd:cornflower', 'xkcd:slate grey'], dodge = False, ax = ax)


# formatting
ax.set_xticks([0, 1], ["Pre-lesion","Post-lesion"], fontsize = 'large', color = 'grey')
ax.tick_params(axis='both', colors='grey')
# ax.set_ylabel()
# ax.legend(False)
ax.set_ylabel('Number of trials in session', fontsize = 'x-large')
ax.set_xlabel('Condition', fontsize = 'x-large')
ax.set_xlim(-0.3, 1.3)


from scipy.stats import ttest_rel
ttest_rel(y[:, 0], y[:, 1])

annot = True
if annot == True:
  # Perform t-tests
  t_stat1, p_val1 = ttest_rel(y[:, 0], y[:, 1])
  # Add annotations
  max_y = max(np.concatenate([y[:, 0], y[:, 1]]))+20

  if p_val1:
      plt.plot([0, 1], [max_y + 0.02, max_y + 0.02], color='black', linewidth=1)
      plt.text(0.5, max_y + 20, f'p={p_val1:.3f}', ha='center', fontsize=10)

sns.despine()
plt.tight_layout()

# Transition matrices

In [8]:
from utils.dfLoading import add_shift_info
sessdf = add_shift_info(sessdf)

c:\Users\dlab\rishika_sim\utils\dfLoading.py:80: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['shift_t0'] = (df['choice_t1']==df['port']).replace({True: 0, False: 1})
c:\Users\dlab\rishika_sim\utils\dfLoading.py:81: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['shift_t1'] = (df['choice_t2']==df['port']).replace({True: 0, False: 1})


In [80]:
# full tm - all trials 
%matplotlib qt
ind = 1
trialsinsess = 100

cond = 'unstr'
an_list = an_dict[cond]
from utils.plotSettings import *
parula = get_parula_cmap()
cmap = parula
mat = np.ones((len(an_list), 4, 4))*np.nan
fig = plt.figure(figsize = (15, 10))
tempdf = sessdf.groupby(['animal','session']).filter(lambda x: x.reward.size >= trialsinsess).groupby(['animal','session']).head(trialsinsess)
mask = (tempdf.task==cond) & tempdf.animal.isin(an_list) & (tempdf.sess_bin>=4)
for animal, group in tempdf[mask].groupby('animal'):

    ax = plt.subplot(6, 8, ind)
    sns.heatmap(pd.crosstab(group['port'],
                            group['choice_t1'],
                            normalize = 'index'),
                            # dropna = False),
                ax = ax, cmap = cmap,
                annot = True, fmt = '.2f',square = True, vmax = 0.17,
                xticklabels = np.arange(1,5), yticklabels = np.arange(1,5),
                # mask = np.eye(4, dtype = bool),
                vmin = 0)
    ax.set_title(animal)
    
    if (animal == 'Yamper') & (group.task.unique()[0] == 'dls'):
        mat_y = np.vstack([pd.crosstab(group['port'],
                            group['choice_t1'],
                            normalize = 'index').to_numpy(), np.zeros((1, 3))*np.nan])
        mat_y = np.hstack([mat_y, np.zeros((4, 1))*np.nan])
        mat[ind-1] = mat_y
    else:
        mat[ind-1] = pd.crosstab(group['port'],
                            group['choice_t1'],
                            normalize = 'index')
    
    ind+=1

plt.tight_layout()
plt.figure()
ax = plt.subplot(111)
sns.heatmap(np.nanmedian(mat, axis = 0), annot = False, fmt = '.2f', cmap = parula, xticklabels = np.arange(1,5), square = True,
            yticklabels = np.arange(1,5), vmin = 0, vmax = 0.17, cbar_kws={'label': 'Transition probability'},
            # mask = np.eye(4),
            ax = ax)
# ax.patch.set_facecolor('white')
ax.set_xlabel('Choice at trial t+1', fontsize = 'x-large')
ax.set_ylabel('Choice at trial t', fontsize = 'x-large')
ax.tick_params(axis = u'both', which = u'both', length = 0, labelsize = 'large', color = 'grey')
plt.title(f'Choice at switches - Animal data, n = {mat.shape[0]}')

# plt.suptitle('Animal data', fontsize = 16)    
plt.tight_layout()
# plt.savefig('C:/Users/dlab/rishika_sim/plots/20231210/tm_all_noswitch.png', dpi = 300)

# Entropy


## Sliding window entropy for different conditions 

In [ ]:
#################################### moving entropy ######################################
%matplotlib qt

trialsinsess = 100

from utils.dfLoading import subset_trials, add_shift_info, add_block_groups
subset_df = add_block_groups(sessdf)
# subset_df = add_shift_info(subset_df)
subset_df = subset_trials(subset_df, trialsinsess=trialsinsess, head_trials = trialsinsess)

from utils.supplementaryFunctions import calc_prob
window = 5
# subset_df['entropy'] = (subset_df.groupby(['animal', 'session'], as_index = False)
#                      .port
#                      .rolling(window, center=True)
#                      .apply(lambda x: entropy(calc_prob(x), base = 2))
#                      .port)

from utils.dfAnalysisHelpers import session_averager
entropy_mean_dict = session_averager(subset_df[subset_df.block_group==1], metric= 'entropy', tasks = ['unstr', 'str', 'dms', 'dls', 'sham'])

In [60]:
# plot average reponse
from utils.plotSettings import *
from scipy.stats import sem
plt.figure(figsize = (5,4.5))
temp_unstr, temp_str = [], []

# set properties
smooth = True
window = 3
label1, label2 = 'Pre-lesion', 'Post-lesion'
# label1, label2 = 'Unstructured', 'Structured'
color1, color2 = 'xkcd:cornflower', 'xkcd:slate grey'
# color1, color2 = 'xkcd:pumpkin orange', 'xkcd:emerald green'
# color1, color2 = '#7a2048', '#408ec6'

# set data for plotting
cond = 'sham'
an_list = an_dict[cond]
# fish = ['Alakazam', 'Yanma']
for (animal, env) in entropy_mean_dict.keys():
    if animal in an_list:
        if env == 'unstr':
            temp_unstr.append(entropy_mean_dict[(animal, env)])
        elif (env == cond):
            temp_str.append(entropy_mean_dict[(animal, env)])

temp_unstr = np.array(temp_unstr)
temp_str = np.array(temp_str)          
if smooth:           
    for i, row in enumerate(temp_unstr):
        temp_unstr[i] = np.convolve(row, np.ones(3)/3, 'same')
        temp_str[i] = np.convolve(temp_str[i], np.ones(3)/3, 'same')
        # temp_dms[i] = np.convolve(temp_dms[i], np.ones(3)/3, 'same')

    # works only for odd windows
    temp_unstr[:, 0:(window//2)] = np.nan
    temp_unstr[:, (trialsinsess-window//2):trialsinsess] = np.nan
    
    temp_str[:, 0:(window//2)] = np.nan
    temp_str[:, (trialsinsess-window//2):trialsinsess] = np.nan
# # 

mean_unstr = (np.mean(np.array(temp_unstr), axis = 0))
mean_str = (np.mean(np.array(temp_str), axis = 0))
# temp_str = (np.mean(np.array(temp_str), axis = 0))

plt.plot(np.arange(trialsinsess), mean_unstr, color1, label = label1) #7a2048
plt.fill_between(np.arange(trialsinsess), mean_unstr+sem(temp_unstr, nan_policy = 'omit'),
                 mean_unstr-sem(temp_unstr, nan_policy = 'omit'), color = color1, alpha = 0.2)

plt.plot(np.arange(trialsinsess), mean_str, color2, label = label2)

plt.fill_between(np.arange(trialsinsess),mean_str+sem(temp_str, nan_policy = 'omit'), 
                 mean_str-sem(temp_str, nan_policy = 'omit'), color = color2, alpha = 0.2)

# pvals = ttest_rel(temp_str, temp_unstr, axis =0 , nan_policy = 'omit')[1]
# print(pvals[1].shape)
# pvals_ind = [ind for ind, i in enumerate(pvals) if i < 0.05]
# plt.scatter(pvals_ind, [0.1]*len(pvals_ind), marker = '.', color = 'r', alpha = 0.4) #(6, 2, 60) for asterisks
# plt.axhline(1.95, color = 'k', linestyle = '--') # max entropy possible
# plt.title('Variation in choice taken')
plt.xlabel('Trials in session', fontsize = 'x-large')
plt.ylabel('Entropy (bits)', fontsize = 'x-large')
plt.xlim(0, trialsinsess)
plt.ylim(0, 0.85)
plt.xticks([0, 25, 50, 75, 100], [1, '', '', '', 100], color = 'grey', fontsize = 'large')
plt.yticks([0, 0.2, 0.4, 0.6, 0.8], [0, '', '', "",0.8], color = 'grey', fontsize = 'large')
plt.title(f'n = {temp_unstr.shape[0]}')
# plt.ylim(0, 0.8)
plt.legend()
sns.despine()
plt.tight_layout()

C:\Users\dlab\AppData\Local\Temp\ipykernel_27724\2479433384.py:48: SmallSampleWarning: After omitting NaNs, one or more axis-slices of one or more sample arguments is too small; corresponding elements of returned arrays will be NaN. See documentation for sample size requirements.
  plt.fill_between(np.arange(trialsinsess), mean_unstr+sem(temp_unstr, nan_policy = 'omit'),
C:\Users\dlab\AppData\Local\Temp\ipykernel_27724\2479433384.py:49: SmallSampleWarning: After omitting NaNs, one or more axis-slices of one or more sample arguments is too small; corresponding elements of returned arrays will be NaN. See documentation for sample size requirements.
  mean_unstr-sem(temp_unstr, nan_policy = 'omit'), color = color1, alpha = 0.2)
C:\Users\dlab\AppData\Local\Temp\ipykernel_27724\2479433384.py:53: SmallSampleWarning: After omitting NaNs, one or more axis-slices of one or more sample arguments is too small; corresponding elements of returned arrays will be NaN. See documentation for sample siz

## Entropy in block 2 and beyond

In [183]:
# for all groups which have a second block, plot first block entropy for first 100 trials of session
# find if 2nd or greater block exists for animals
block_2_exists = tempdf[(tempdf.block_group>=2) & (tempdf.task == 'unstr')].groupby(['animal', 'sess_block']).size().reset_index()[['animal', 'sess_block']]

# remove animals with less than 10
block_2_exists = block_2_exists[block_2_exists.animal.isin(block_2_exists.animal.value_counts()[block_2_exists.animal.value_counts()>10].index)]

# find first blocks for these sessions
all_blocked_groups = tempdf[tempdf.task == 'unstr'].merge(block_2_exists, on = ['animal', 'sess_block'])
first_blocks = all_blocked_groups[all_blocked_groups.block_group == 1]
other_blocks = all_blocked_groups[all_blocked_groups.block_group > 1]

# convert to numpy array
entropy2_dict_block_1 = {}
for an, group in first_blocks.groupby('animal'):
        g = group.groupby('session').cumcount()
        L = (group.set_index(['session', g])
                .unstack(fill_value=0)
                .stack(future_stack= True).groupby(level=0)
                .apply(lambda x: x.entropy.values.tolist())
                .tolist())
        entropy2_dict_block_1[an] = np.mean(L, axis = 0)
entropy2_dict_block_others = {}
for an, group in other_blocks.groupby('animal'):
        g = group.groupby('session').cumcount()
        L = (group.set_index(['session', g])
                .unstack(fill_value=0)
                .stack(future_stack= True).groupby(level=0)
                .apply(lambda x: x.entropy.values.tolist())
                .tolist())
        entropy2_dict_block_others[an] = np.mean(L, axis = 0)


In [204]:
plt.figure(figsize = (5, 4.5))
prev_sess_size = 25
y = np.zeros((len(entropy2_dict_block_1.keys()), 100+prev_sess_size))
for ind, an in enumerate(entropy2_dict_block_1.keys()):
    y[ind] = np.concatenate([entropy2_dict_block_1[an][-prev_sess_size:], entropy2_dict_block_others[an]])
    plt.plot(y[ind], color = 'xkcd:cornflower', alpha = 0.3, lw = 0.75)
plt.plot(np.nanmean(y, axis = 0), color = 'xkcd:cornflower', lw = 2.25)
plt.xlabel('Block position', fontsize = 'x-large')
plt.ylabel('Entropy (bits)', fontsize = 'x-large')
plt.ylim(0, 0.4)
plt.xticks(np.arange(0, 126, 25), np.arange(-25, 101, 25), color = 'grey', fontsize = 'large')
plt.axvline(25, lw = 0.75, color = 'k', linestyle = '--')
plt.yticks(color = 'grey', fontsize = 'large')
sns.despine()
plt.tight_layout()

C:\Users\dlab\AppData\Local\Temp\ipykernel_19448\2989309347.py:7: RuntimeWarning: Mean of empty slice
  plt.plot(np.nanmean(y, axis = 0), color = 'xkcd:cornflower', lw = 2.25)


# Regret

In [16]:
#################################### regret ######################################
%matplotlib qt

trialsinsess = 100

from utils.dfLoading import subset_trials, add_shift_info, add_block_groups
subset_df = add_block_groups(sessdf)
# subset_df = add_shift_info(df)
subset_df = subset_trials(subset_df, trialsinsess=trialsinsess, head_trials = trialsinsess)

from utils.supplementaryFunctions import calc_prob
window = 5
subset_df['regret'] = abs(subset_df['rewprob'] - 80)


from utils.dfAnalysisHelpers import session_averager
reg_mean_dict = session_averager(subset_df[subset_df.block_group == 1], metric= 'regret', tasks = ['unstr', 'str', 'dms', 'dls', 'sham'])

In [ ]:
%matplotlib qt
from scipy.stats import sem
# plot average reponse
plt.figure(figsize = (5,4.5))
temp_unstr, temp_str = [], []

# set properties
smooth = True
window = 3
label1 = 'Pre-lesion'
label2 = 'Post-lesion'
# label1 = 'Unstructured'
# label2 = 'Structured'
color1 = 'xkcd:cornflower'
color2 = 'xkcd:slate grey'
# color1 = '#7a2048', 
# color1 = 'xkcd:pumpkin orange'
# color2 = 'xkcd:emerald green'
# color2 = '#408ec6'

# set data for plotting
cond = 'dms'
for (animal, env) in reg_mean_dict.keys():
    if animal in an_dict[cond]:
        if env == 'unstr':
            temp_unstr.append(reg_mean_dict[(animal, env)])
        elif (env == cond):
            temp_str.append(reg_mean_dict[(animal, env)])

# calculate performance index
temp_unstr = (1-np.array(temp_unstr)/100)
temp_str = (1-np.array(temp_str)/100)
# temp_unstr = np.array(temp_unstr)
# temp_str = np.array(temp_str)

# smoothing details
if smooth:           
    for i, row in enumerate(temp_unstr):
        temp_unstr[i] = np.convolve(row, np.ones(window)/window, 'same')
        # temp_str[i] = np.convolve(temp_str[i], np.ones(window)/window, 'same')
    
    # works only for odd windows
    temp_unstr[:, 0:(window//2)] = np.nan
    temp_unstr[:, (trialsinsess-window//2):trialsinsess] = np.nan
    
    temp_str[:, 0:(window//2)] = np.nan
    temp_str[:, (trialsinsess-window//2):trialsinsess] = np.nan

# temp_str = [np.convolve(temp_str[i], np.ones(window)/window, 'same') for i, row in enumerate(temp_str)]
# temp_str[:, 0:(window//2)] = np.nan
# temp_str[:, (trialsinsess-window//2):trialsinsess] = np.nan

# take mean of smoothed data   
mean_unstr = (np.mean(np.array(temp_unstr), axis = 0))
mean_str = (np.mean(np.array(temp_str), axis = 0))

# plot line 1
plt.plot(np.arange(trialsinsess), mean_unstr, color1, label = label1) 
plt.fill_between(np.arange(trialsinsess), mean_unstr+sem(temp_unstr, nan_policy = 'omit'),
                 mean_unstr-sem(temp_unstr, nan_policy = 'omit'), color = color1, alpha = 0.2)

# plot line 2 (or comment)
plt.plot(np.arange(trialsinsess), mean_str, color2, label = label2)
plt.fill_between(np.arange(trialsinsess), mean_str+sem(temp_str, nan_policy = 'omit'), 
                 mean_str-sem(temp_str, nan_policy = 'omit'), color = color2, alpha = 0.2)
# for row in temp_str:
#     plt.plot(row, color = color2, alpha  = 1)

# plot significance
# pvals = ttest_rel(temp_str, temp_unstr)[1]
# pvals_ind = [ind for ind, i in enumerate(pvals) if i < 0.05]
# plt.scatter(pvals_ind, [0.65]*len(pvals_ind), marker = '.', color = 'r', alpha = 0.4) #(6, 2, 60) for asterisks

# plot settings
plt.xlabel('Trials in session', fontsize = 'x-large')
plt.ylabel('Performance index', fontsize = 'x-large')
plt.xlim(0, trialsinsess)
plt.axhline(0.625, color='k', linestyle = '--') # chance calc by
plt.ylim(0.6, 0.8)
plt.xticks([0, 25, 50, 75, 100], [1, '', '', '', 100], color = 'grey', fontsize = 'large')
plt.yticks([0.6, 0.65, 0.7, 0.75, 0.8, 0.85], ['0.60', '', '', '', '', 0.85], color = 'grey', fontsize = 'large')
# plt.ylim(0, 0.8)
# plt.legend(loc='upper left')
plt.title(f'n = {temp_unstr.shape[0]}')
sns.despine()
# sns.set_context('talk')
plt.tight_layout()
# plt.savefig('C:/Users/dlab/OneDrive - Indian Institute of Science/Drawings/awp1/entropy_n_21.png', dpi = 300)

C:\Users\dlab\AppData\Local\Temp\ipykernel_26240\1611805775.py:63: SmallSampleWarning: After omitting NaNs, one or more axis-slices of one or more sample arguments is too small; corresponding elements of returned arrays will be NaN. See documentation for sample size requirements.
  plt.fill_between(np.arange(trialsinsess), mean_unstr+sem(temp_unstr, nan_policy = 'omit'),
C:\Users\dlab\AppData\Local\Temp\ipykernel_26240\1611805775.py:64: SmallSampleWarning: After omitting NaNs, one or more axis-slices of one or more sample arguments is too small; corresponding elements of returned arrays will be NaN. See documentation for sample size requirements.
  mean_unstr-sem(temp_unstr, nan_policy = 'omit'), color = color1, alpha = 0.2)
C:\Users\dlab\AppData\Local\Temp\ipykernel_26240\1611805775.py:68: SmallSampleWarning: After omitting NaNs, one or more axis-slices of one or more sample arguments is too small; corresponding elements of returned arrays will be NaN. See documentation for sample siz

In [609]:
# if fitting exp curve is reqd
def logF(x, a, b, c):
    return a*np.log(x+b)+c

def f(x, a, b, c):
    '''exponential fxn: a is amplitude, b is decay rate, c is offset'''
    return a*np.exp(-b*x) + c

popt_str, _ = curve_fit(f, np.arange(trialsinsess), mean_str, nan_policy = 'omit')
popt_unstr, _ = curve_fit(f, np.arange(trialsinsess), mean_unstr, nan_policy = 'omit')
plt.plot(np.arange(trialsinsess), f(np.arange(trialsinsess), *popt_str), color2, linestyle= 'dotted')
plt.plot(np.arange(trialsinsess), f(np.arange(trialsinsess), *popt_unstr), color1, linestyle= 'dotted')

In [ ]:
# logic - if 46 next to 80 then structured, but also how visibly smooth it is
l_str_sess = [
    '[10 15 46 80]',
    '[10 15 80 46]',
    '[10 46 80 15]',
    '[10 80 46 15]',
    '[15 10 46 80]',
    '[15 10 80 46]', 
    '[15 46 46 80]', 
    '[15 46 80 10]',
    '[15 46 80 46]',
    '[15 80 46 10]',
    '[15 80 46 46]',
    '[46 46 80 15]',
    '[46 80 10 15]',
    '[46 80 15 10]',
    '[46 80 46 15]',
    '[80 46 10 15]',
    '[80 46 15 10]',
    '[80 46 46 15]'
]
sessdf['assigned'] = sessdf.rewprobfull.isin(l_str_sess)
sessdf.assigned.replace({True: 'str', False: 'unstr'}, inplace = True)

struc = ['[80 46 15 10]', '[46 80 46 15]', '[15 46 80 46]', '[10 15 46 80]']
sessdf['env'] = sessdf.rewprobfull.isin(struc)
sessdf.env.replace({True: 'str', False: 'unstr'}, inplace = True)

In [316]:
%matplotlib qt
from scipy.signal import find_peaks
plt.figure(figsize = (20, 10))
unq_prob, _ = np.array(np.unique(sessdf[(sessdf.animal == 'Blissey')].rewprobfull,
          return_counts = True))

for i in range(unq_prob.shape[0]):
    ax = plt.subplot(4, 9, i+1)
    rp = [int(o) for o in unq_prob[i]]
    pk = find_peaks(rp, height = 10)
    ax.plot(rp)
    ax.set_title(f'{pk[0].shape[0]}, {pk[1]["peak_heights"]}')
plt.tight_layout()

# Reward rate analysis

In [8]:
#################################### rr ######################################
# set_cwd('/home/rishika/sim/')
%matplotlib qt
# %matplotlib inline

trialsinsess = 100

from utils.dfLoading import subset_trials, add_shift_info, add_block_groups
subset_df = add_block_groups(sessdf)
# subset_df = add_shift_info(df)
subset_df = subset_trials(subset_df, trialsinsess=trialsinsess, head_trials = trialsinsess)

# window = 
# subset_df['rr'] = subset_df.groupby(['animal', 'session'], as_index = False).reward.rolling(window, center=True).mean().reward


from utils.dfAnalysisHelpers import session_averager
rr_mean_dict = session_averager(subset_df[subset_df.block_group == 1], metric= 'rr', tasks = ['unstr', 'str', 'dms', 'dls', 'sham'])

In [95]:
from scipy.optimize import curve_fit
from scipy.stats import sem
from itertools import permutations
# plot average reponse
from utils.plotSettings import *
plt.figure(figsize = (5,4.5))
temp_unstr, temp_str = [], []

# set properties
smooth = False
window = 3
label1, label2 = 'Pre-lesion', 'Post-lesion'
# label1, label2 = 'Unstructured', 'Structured'
color1, color2 = 'xkcd:cornflower', 'xkcd:slate grey'
# color1, color2 = 'xkcd:pumpkin orange', 'xkcd:emerald green'
# # color1, color2 = '#7a2048', '#408ec6'

# set data for plotting
cond = 'unstr'
for (animal, env) in rr_mean_dict.keys():
    if animal in an_dict[cond]:
        if env == 'unstr':
            temp_unstr.append(rr_mean_dict[(animal, env)])
        # elif (env == cond):
        #     temp_str.append(rr_mean_dict[(animal, env)])
temp_unstr = np.array(temp_unstr)
# temp_str = np.array(temp_str)

# smoothing details
if smooth == True:           
    for i, row in enumerate(temp_unstr):
        temp_unstr[i] = np.convolve(row, np.ones(window)/window, 'same')
        # temp_str[i] = np.convolve(temp_str[i], np.ones(window)/window, 'same')
    
    temp_unstr = np.array(temp_unstr)
    # temp_str = np.array(temp_str)
    # 
    # works only for odd windows
    temp_unstr[:, 0:(window//2)] = np.nan
    temp_unstr[:, (trialsinsess-window//2):trialsinsess] = np.nan
    
    # temp_str[:, 0:(window//2)] = np.nan
    # temp_str[:, (trialsinsess-window//2):trialsinsess] = np.nan

# take mean of smoothed data   
mean_unstr = (np.nanmean(np.array(temp_unstr), axis = 0))
mean_str = (np.nanmean(np.array(temp_str), axis = 0))

# plot line 1
plt.plot(np.arange(trialsinsess), mean_unstr, color1, label = label1)#, marker = '.') 
plt.fill_between(np.arange(trialsinsess), mean_unstr+sem(temp_unstr, nan_policy = 'omit'),
                 mean_unstr-sem(temp_unstr, nan_policy = 'omit'), color = color1, alpha = 0.2)

# plot line 2 (or comment)
# plt.plot(np.arange(trialsinsess), mean_str, color2, label = label2)
plt.fill_between(np.arange(trialsinsess), mean_str+sem(temp_str, nan_policy = 'omit'), 
                 mean_str-sem(temp_str, nan_policy = 'omit'), color = color2, alpha = 0.2)

# plot significance
# pvals = ttest_rel(temp_str, temp_unstr)[1]
# pvals_ind = [ind for ind, i in enumerate(pvals) if i < 0.05]
# plt.scatter(pvals_ind, [0.45]*len(pvals_ind), marker = '.', color = 'r', alpha = 0.4) #(6, 2, 60) for asterisks

# chance calc
plt.axhline(np.mean(np.unique(list(permutations([80, 46, 15, 10]))+list(permutations([46, 80, 46, 15])), axis = 0))/100,
            color = 'k', linestyle = '--')
# plt.axhline(np.mean([[80, 46, 15, 10], [46, 80, 46, 15], [15, 46, 80, 46], [10, 15, 46, 80]])/100, color = color2)

# plot settings
plt.xlabel('Trials in session', fontsize = 'x-large')
plt.ylabel('Reward rate', fontsize = 'x-large')
plt.xlim(0, trialsinsess)
plt.ylim(0.4, 0.7)
plt.xticks([0, 25, 50, 75, 100], [1, '', '', '', 100], color = 'grey', fontsize = 'large')
plt.yticks([0.4, 0.5, 0.6, 0.7], [0.4, '', '', 0.7], color = 'grey', fontsize = 'large')
plt.title(f'n = {temp_unstr.shape[0]}')
plt.legend(loc = 'upper left')
sns.despine()
# sns.set_context('talk')
plt.tight_layout()
# plt.savefig('C:/Users/dlab/OneDrive - Indian Institute of Science/Drawings/awp1/entropy_n_21.png', dpi = 300)


# if fitting exp curve is reqd
def logFunc(x, a, b):
    return a + b*np.log(x)

# popt_str, _ = curve_fit(logFunc, np.arange(150), temp_str, nan_policy = 'omit')
# popt_unstr, _ = curve_fit(logFunc, np.arange(trialsinsess), mean_unstr, nan_policy = 'omit')
# plt.plot(np.arange(150), logFunc(np.arange(150), *popt_str), '#408ec6', linestyle= 'dotted')
# plt.plot(np.arange(trialsinsess), logFunc(np.arange(trialsinsess), *popt_unstr), color1, linestyle= 'dotted')

C:\Users\dlab\AppData\Local\Temp\ipykernel_27724\1329893560.py:46: RuntimeWarning: Mean of empty slice
  mean_unstr = (np.nanmean(np.array(temp_unstr), axis = 0))
C:\Users\dlab\AppData\Local\Temp\ipykernel_27724\1329893560.py:47: RuntimeWarning: Mean of empty slice
  mean_str = (np.nanmean(np.array(temp_str), axis = 0))
C:\Users\dlab\AppData\Local\Temp\ipykernel_27724\1329893560.py:51: SmallSampleWarning: After omitting NaNs, one or more axis-slices of one or more sample arguments is too small; corresponding elements of returned arrays will be NaN. See documentation for sample size requirements.
  plt.fill_between(np.arange(trialsinsess), mean_unstr+sem(temp_unstr, nan_policy = 'omit'),
C:\Users\dlab\AppData\Local\Temp\ipykernel_27724\1329893560.py:52: SmallSampleWarning: After omitting NaNs, one or more axis-slices of one or more sample arguments is too small; corresponding elements of returned arrays will be NaN. See documentation for sample size requirements.
  mean_unstr-sem(temp_u

In [288]:
temp_unstr, temp_str = [], []
for (animal, env) in rr_mean_dict.keys():
    if animal in fish:
        if env == 'unstr':
            temp_unstr.append(rr_mean_dict[(animal, env)])
        elif env == 'str':
            temp_str.append(rr_mean_dict[(animal, env)])

ttest_rel(np.nanmean(np.array(temp_str)[:, :25], axis = 1),
          np.nanmean(np.array(temp_str)[:, -25:], axis = 1), nan_policy = 'omit')

plt.figure(figsize = (3.5,4.5))
rr_df = pd.DataFrame([np.nanmean(np.array(temp_str), axis = 1), np.nanmean(np.array(temp_unstr), axis = 1)],
                     index = ['Structured', 'Unstructured'])
plt.scatter(np.zeros(5), rr_df.loc['Structured'], color = '#408ec6', alpha = 0.5)
plt.scatter(np.ones(5), rr_df.loc['Unstructured'], color = '#7a2048', alpha = 0.5)
plt.plot(rr_df, '-', c = 'grey', alpha = 0.5)
sns.pointplot(rr_df.T, join = False, palette = ['#408ec6','#7a2048'], errorbar = None)
sns.despine()
plt.xlim([-0.4,1.4])
# ax.set_xticks([0,1])
plt.xlabel('Session')
# plt.xticks([0,1],['Structured', 'End'])
plt.ylabel('Reward rate')
plt.title('Reward rate across conditions', wrap = True)
plt.tight_layout()

In [97]:
# plot 
window = 7
trialsinsess = 100
fig = plt.figure(figsize=(10,10))
rr_mean_dict_unstr = {}
# rem_les = sessdf[~(sessdf.task.isin(['dms', 'dls', 'sham']))]
fish = ['test05022023', 'Blissey', 'Chikorita', 'Darkrai', 'Eevee']

colors ={'str':'#1e2761', 'unstr':'#7a2048', 'dls':'xkcd:gold', 'dms':'xkcd:red', 'sham':'b', 'dms_str':'xkcd:orange'} 
for i in ['unstr', 'str']:

    for ind, (animal, group) in enumerate(sessdf[sessdf.animal.isin(fish)].groupby('animal')):
        ax = plt.subplot(4, 7, ind+1)
        
        # remove duplicates - if a session has duplicates remove it 
        # group = group.loc[~(sessdf.groupby('animal').get_group(animal)
        #                           .duplicated(subset = 'trialstart', keep = False))]
        
        # filter by sessions>= length
        filtered = (group[group.env.isin(['str']) & (group.task.isin([i]))]
                    .groupby('session')).filter(lambda x: x.reward.size >= trialsinsess)

        # take only those sessions
        filtered = filtered.groupby('session').head(trialsinsess)
        if filtered.empty==True:
            continue
            
        # convert to numpy array
        g = filtered.groupby('session').cumcount()
        L = np.array(filtered.set_index(['session',g])
               .unstack(fill_value=0)
               .stack().groupby(level=0)
               .apply(lambda x: x.rr.values.tolist())
               .tolist())
        
        print(L.shape, animal, i)
        # mean
        rr_mean = np.mean(L, axis=0)
        # reg_sem = np.std(L)
        rr_mean_dict_unstr[animal, i] = rr_mean
        ax.plot(rr_mean, label = i, c = colors[i])
        # ax.fill_between(np.arange(trialsinsess), reg_sem+reg_mean, reg_mean-reg_sem, color = colors[i], alpha = 0.2)
        ax.set_title(animal)
        # ax.legend(['Pure unstructured','__', 'Structured (in unstr)'])
        ax.set_xlabel('Trial')

sns.despine()
fig.supylabel('Reward rate, averaged across sessions')
plt.tight_layout()

(49, 100) Blissey unstr
(17, 100) Chikorita unstr
(19, 100) Darkrai unstr
(9, 100) Eevee unstr
(18, 100) test05022023 unstr
(177, 100) Blissey str
(136, 100) Chikorita str
(65, 100) Darkrai str
(209, 100) Eevee str
(104, 100) test05022023 str


In [106]:
# plot average reponse and add structured in structured also
plt.figure(figsize = (5,4.5))
temp_unstr, temp_str, temp_str_str = [], [], []

# set properties
smooth = True
window = 3
label1 = 'Structured (in unstr)'
label2 = 'Structured (in str)'
# label3 = 'Structured (in str)'
color1 = 'xkcd:azure'
color2 = 'xkcd:slate grey'
# color1 = '#7a2048', 
color1 = 'xkcd:olive'
color2 = 'xkcd:emerald green'
# color3 = 'xkcd:magenta'
# color2 = '#408ec6'

# set data for plotting
# fish = ['Blissey', 'Darkrai', 'Mesprit', 'Quilava', 'Raltz', 'Inkay']
# fish = ['Hoppip', 'Kirlia', 'Nidorina', 'Togepi']
# fish = ['test05022023', 'Jirachi', 'Goldeen', 'Phione']
fish = ['test05022023', 'Blissey', 'Chikorita', 'Darkrai', 'Eevee']
for (animal, env) in rr_mean_dict_unstr.keys():
    if animal in fish:
        if env == 'unstr':
            temp_unstr.append(rr_mean_dict_unstr[(animal, env)])
        elif (env == 'str'):
            temp_str.append(rr_mean_dict_unstr[(animal, env)])
            # temp_str_str.append(rr_mean_dict[(animal, env)])

# calculate performance index
# temp_unstr = (1-np.array(temp_unstr)/100)
# temp_str = (1-np.array(temp_str)/100)
temp_unstr = np.array(temp_unstr)
temp_str = np.array(temp_str)
# temp_str_str = np.array(temp_str_str)

# smoothing details
if smooth:           
    for i, row in enumerate(temp_unstr):
        temp_unstr[i] = np.convolve(row, np.ones(window)/window, 'same')
        temp_str[i] = np.convolve(temp_str[i], np.ones(window)/window, 'same')
        # temp_str_str[i] = np.convolve(temp_str_str[i], np.ones(window)/window, 'same')
    # 
    # works only for odd windows
    temp_unstr[:, 0:(window//2)] = np.nan
    temp_unstr[:, (trialsinsess-window//2):trialsinsess] = np.nan
    
    temp_str[:, 0:(window//2)] = np.nan
    temp_str[:, (trialsinsess-window//2):trialsinsess] = np.nan

    # temp_str_str[:, 0:(window//2)] = np.nan
    # temp_str_str[:, (trialsinsess-window//2):trialsinsess] = np.nan

# take mean of smoothed data   
mean_unstr = (np.mean(np.array(temp_unstr), axis = 0))
mean_str = (np.mean(np.array(temp_str), axis = 0))
# mean_str_str = (np.mean(np.array(temp_str_str), axis = 0))

# plot line 1
plt.plot(np.arange(trialsinsess), mean_unstr, color1, label = label1) 
plt.fill_between(np.arange(trialsinsess), mean_unstr+sem(temp_unstr, nan_policy = 'omit'),
                 mean_unstr-sem(temp_unstr, nan_policy = 'omit'), color = color1, alpha = 0.2)

# plot line 2 (or comment)
plt.plot(np.arange(trialsinsess), mean_str, color2, label = label2)
plt.fill_between(np.arange(trialsinsess), mean_str+sem(temp_str, nan_policy = 'omit'), 
                 mean_str-sem(temp_str, nan_policy = 'omit'), color = color2, alpha = 0.2)

# plot line 3 (or comment)
# plt.plot(np.arange(trialsinsess), mean_str_str, color3, label = label3)
# plt.fill_between(np.arange(trialsinsess), mean_str_str+sem(temp_str_str, nan_policy = 'omit'), 
#                  mean_str_str-sem(temp_str_str, nan_policy = 'omit'), color = color3, alpha = 0.2)

# plot significance
# pvals = ttest_rel(temp_str, temp_unstr)[1]
# pvals_ind = [ind for ind, i in enumerate(pvals) if i < 0.05]
# plt.scatter(pvals_ind, [0.65]*len(pvals_ind), marker = '.', color = 'r', alpha = 0.4) #(6, 2, 60) for asterisks

# plot settings
plt.xlabel('Trials in session', fontsize = 'x-large')
plt.ylabel('Reward rate', fontsize = 'x-large')
plt.xlim(0, trialsinsess)
# plt.axhline(0.625, color='k', linestyle = '--') # chance calc by
# plt.ylim(0.6, 0.95)
plt.xticks([0, 25, 50, 75, 100], [1, '', '', '', 100], color = 'grey', fontsize = 'large')
plt.ylim(0.4, 0.7)

plt.yticks([0.4, 0.5, 0.6, 0.7], [0.4, '', '', 0.7], color = 'grey', fontsize = 'large')

plt.legend(loc='upper left')
sns.despine()
# sns.set_context('talk')
plt.tight_layout()
# plt.savefig('C:/Users/dlab/OneDrive - Indian Institute of Science/Drawings/awp1/entropy_n_21.png', dpi = 300)

# Bias analysis

In [53]:
# stolen from Sanika
#plot number of times each port was chosen/total trials in block as a function of rewprob at that port, for all trials indf; average across blocks
port_choices = [1, 2, 3, 4]
#group by unique rewprob values, find unique rewprob values across all ports first
colors_ports = ['red', 'green', 'blue', 'purple']
unique_rewprobs = np.unique(sessdf['rewprob'])
# print(unique_rewprobs)
#remove nan from unique_rewprobs
unique_rewprobs = unique_rewprobs[~np.isnan(unique_rewprobs)]
# print(unique_rewprobs)
#remove rows with nan rewprob
# sessdf = sessdf[~sessdf['rewprob'].isna()]
plt.figure(figsize=(20, 10))

cond = 'dls'
mask = (sessdf.task == cond) & (sessdf.animal.isin(an_dict[cond])) #& (sessdf.sess_bin >= 4)

nanimals = sessdf[mask].animal.nunique()
pn = 1

for animal, group in sessdf[mask].groupby('animal'):
    #for every block, calculate number of times each port was chosen/total trials in block for each unique rewprob value
    nblocks = group['session'].nunique()
    port_choice_matrix = np.zeros((len(unique_rewprobs), len(port_choices), nblocks))
    #for all blocks
    #save all rewprobfull per block in a list
    for block in range(1, group['session'].nunique()+1):
        block_data = group[group['session'] == block]
        #for all ports
        for p, port in enumerate(port_choices):
            port_data = block_data[block_data['port'] == port]
            #for all unique rewprobs
            for r, rewprob in enumerate(unique_rewprobs):
                n_choices = len(port_data[port_data['rewprob'] == rewprob])
                port_choice_matrix[r, p, block-1] = n_choices

    bias_matrix = np.zeros((port_choice_matrix.shape[0], port_choice_matrix.shape[1]))
    counter = np.zeros((port_choice_matrix.shape[0], port_choice_matrix.shape[1]))

    for p in range(port_choice_matrix.shape[1]):
        for r in range(port_choice_matrix.shape[0]):
            bias_matrix[r, p] = 0
            for b in range(port_choice_matrix.shape[2]):
                ntrls = np.sum(port_choice_matrix[:,:,b])
                if ntrls != 0:
                    bias_matrix[r, p] += port_choice_matrix[r, p, b] / ntrls
                if port_choice_matrix[r, p, b] > 0:
                    counter[r, p] += 1

    #divide fin_mat by counter
    for p in range(port_choice_matrix.shape[1]):
        for r in range(port_choice_matrix.shape[0]):
            if counter[r, p] > 0:
                bias_matrix[r, p] = bias_matrix[r, p] / counter[r, p]
# print('Bias matrix:', bias_matrix)

    ax = plt.subplot(6, 8, pn)
    #plot as line plots for each port
    for p, port in enumerate(port_choices):
        ax.plot([0,1,2,3], bias_matrix[:, p], label=f'Port {port}', marker='o', linewidth=2, color=colors_ports[p])
    # plt.xlabel('Reward Probability')
    ax.set_xticks([0,1,2,3], labels=[int(r) for r in unique_rewprobs])
    # plt.ylabel(f'Fraction chosen (averaged across {nblocks} blocks)')
    ax.set_title(f'{animal}')
    pn+=1

plt.tight_layout()
    # n_rows, n_cols = bias_matrix.shape
    # rpf = [10,15,46,80]
    # labels = [i+1 for i in range(n_cols)]

    # # Angles for radar
    # angles = np.linspace(0, 2*np.pi, n_cols, endpoint=False)
    # angles = np.concatenate([angles, [angles[0]]])  # close loop

    # fig, ax = plt.subplots(figsize=(6, 6), subplot_kw=dict(polar=True))
    # for i in range(n_rows):
    #     values = bias_matrix[i]
    #     values = np.concatenate([values, [values[0]]])  # close loop

    #     ax.plot(angles, values, linewidth=2, label=f'prob {rpf[i]}', color=colors_ports[i])
    #     # ax.fill(angles, values, alpha=0.15)

    # ax.set_thetagrids(np.degrees(angles[:-1]), labels)
    # #reduce the yticks to 3
    # # ax.set_yticks([])
    # ax.legend(loc='upper right', bbox_to_anchor=(1.25, 1.1))
    # plt.show()

# predict avg performance based on reward probability - 25-09-2024

In [54]:
#################################### perf grouped by rp ######################################
# set_cwd('/home/rishika/sim/')
%matplotlib qt
window = 7
trialsinsess = 100
fig = plt.figure(figsize=(20,10))
# sessdf['rr'] = sessdf.groupby(['animal', 'session'], as_index = False).reward.rolling(window, center=True).mean().reward
rr_mean_dict_rp = {}
# rem_les = sessdf[~(sessdf.task.isin(['dms', 'dls', 'sham']))]

colors ={'str':'#028f1e55', 'unstr':'#fb7d0755', 'dls':'xkcd:gold', 'dms':'xkcd:red', 'sham':'b', 'ds':'xkcd:orange'} 
for i in ['unstr']:

    for ind, (rpfull_string, group) in enumerate(sessdf.groupby('rewprobfull')):
        ax = plt.subplot(5, 8,  ind+1)
        
        # remove duplicates
        # if animal == 'test05022023':
        #     group = group.loc[~(sessdf.groupby('animal').get_group('test05022023')
        #                               .duplicated(subset = 'trialstart', keep = False))]
        
        # filter by sessions>= length
        filtered = (group[group.task.isin([i])]
                    .groupby(['animal','session'])).filter(lambda x: x.reward.size >= trialsinsess)

        # take only those sessions
        filtered = filtered.groupby(['animal','session']).head(trialsinsess)
        
        # if animal=='Nidorina':
        #     filtered = filtered[filtered.datetime.astype('datetime64[ns]')<datetime.datetime(2024, 5, 8)]
        
        if filtered.empty==True:
            continue

        rp_type = filtered.assigned.iloc[0]
        
        # convert to numpy array
        g = filtered.groupby(['animal','session']).cumcount()
        L = np.array(filtered.set_index(['animal','session',g])
               .unstack(fill_value=0)
               .stack().groupby(level = ['animal', 'session'])
               .apply(lambda x: x.rr.values.tolist())
               .tolist())
        
        # mean
        rr_mean = np.mean(L, axis=0)
        rr_mean_dict_rp[rpfull_string, rp_type] = rr_mean
        ax.plot(rr_mean, label = i, c = 'k')
        ax.set_title(rpfull_string.strip('[]'))
        # ax.legend()
        ax.set_facecolor(colors[rp_type])
        # ax.set_xlabel('Trial')
        ax.axhline(np.array(rpfull_string.strip('[]').split(), dtype = int).mean()/100, linestyle = '--', color = 'k')
        ax.set_ylim(0.3, 0.8)

sns.despine()
fig.supylabel('Reward rate, averaged across sessions')
plt.tight_layout()

AttributeError: 'DataFrame' object has no attribute 'assigned'

In [164]:
ax = plt.subplot(111)

dat = pd.DataFrame(rr_mean_dict_rp).mean().sort_values().reset_index()
dat.rename(columns = {'level_0':'rp', 'level_1':'rp_type', 0:'mu'}, inplace = True)
sns.barplot(dat, x = 'rp', y = 'mu', hue = 'rp_type', palette = [color1, color2])
plt.xticks(rotation=90, ha='right')
plt.tight_layout()
sns.despine()

In [165]:
ax = plt.subplot(111)

dat = pd.DataFrame(rr_mean_dict_rp).head(50).mean().sort_values().reset_index()
dat.rename(columns = {'level_0':'rp', 'level_1':'rp_type', 0:'mu'}, inplace = True)
sns.barplot(dat, x = 'rp', y = 'mu', hue = 'rp_type', palette = [color2, color1])
plt.xticks(rotation=90, ha='right')
plt.tight_layout()
plt.title('first 50')
sns.despine()

In [168]:
ax = plt.subplot(111)

dat = pd.DataFrame(rr_mean_dict_rp).tail(50).mean().sort_values().reset_index()
dat.rename(columns = {'level_0':'rp', 'level_1':'rp_type', 0:'mu'}, inplace = True)
sns.barplot(dat, x = 'rp', y = 'mu', hue = 'rp_type', palette = [color1, color2])
plt.xticks(rotation=90, ha='right')
plt.tight_layout()
plt.title('last 50')
sns.despine()

C:\Users\dlab\miniforge3\envs\rishika\lib\site-packages\ipykernel\eventloops.py:145: UserWarning: The figure layout has changed to tight
  el.exec() if hasattr(el, "exec") else el.exec_()


In [170]:
ax = plt.subplot(111)

dat = pd.DataFrame(rr_mean_dict_rp).tail(15).max().sort_values().reset_index()
dat.rename(columns = {'level_0':'rp', 'level_1':'rp_type', 0:'mu'}, inplace = True)
sns.barplot(dat, x = 'rp', y = 'mu', hue = 'rp_type', palette = [color1, color2])
plt.xticks(rotation=90, ha='right')
plt.tight_layout()
plt.title('max of last 15')
sns.despine()

C:\Users\dlab\miniforge3\envs\rishika\lib\site-packages\ipykernel\eventloops.py:145: UserWarning: The figure layout has changed to tight
  el.exec() if hasattr(el, "exec") else el.exec_()


In [172]:
ax = plt.subplot(111)

dat = pd.DataFrame(rr_mean_dict_rp).median().sort_values().reset_index()
dat.rename(columns = {'level_0':'rp', 'level_1':'rp_type', 0:'mu'}, inplace = True)
sns.barplot(dat, x = 'rp', y = 'mu', hue = 'rp_type', palette = [color2, color1])
plt.xticks(rotation=90, ha='right')
plt.tight_layout()
plt.title('median')
sns.despine()

C:\Users\dlab\miniforge3\envs\rishika\lib\site-packages\ipykernel\eventloops.py:145: UserWarning: The figure layout has changed to tight
  el.exec() if hasattr(el, "exec") else el.exec_()


In [89]:
# plot average reponse
plt.figure(figsize = (5,4.5))
temp_unstr, temp_str = [], []

# set properties
smooth = True
window = 3
label1 = 'Unstructured'
label2 = 'Structured'
color1 = 'xkcd:azure'
color2 = 'xkcd:slate grey'
# color1 = '#7a2048', 
color1 = 'xkcd:pumpkin orange'
color2 = 'xkcd:emerald green'
# color2 = '#408ec6'

# set data for plotting
# fish = ['Blissey', 'Darkrai', 'Mesprit', 'Quilava', 'Raltz', 'Inkay']
# fish = ['Hoppip', 'Kirlia', 'Nidorina', 'Togepi']
# fish = ['test05022023', 'Jirachi', 'Goldeen', 'Phione']
fish = ['test05022023', 'Blissey', 'Chikorita', 'Darkrai', 'Eevee']
for (animal, env) in rr_mean_dict_rp.keys():
    # if animal in fish:
    if env == 'unstr':
        temp_unstr.append(rr_mean_dict_rp[(animal, env)])
    elif (env == 'str'):
        temp_str.append(rr_mean_dict_rp[(animal, env)])

# calculate performance index
temp_unstr = np.array(temp_unstr)
temp_str = np.array(temp_str)

# smoothing details
if smooth == True:           
    for i, row in enumerate(temp_unstr):
        temp_unstr[i] = np.convolve(row, np.ones(window)/window, 'same')
        temp_str[i] = np.convolve(temp_str[i], np.ones(window)/window, 'same')
    
    # works only for odd windows
    temp_unstr[:, 0:(window//2)] = np.nan
    temp_unstr[:, (trialsinsess-window//2):trialsinsess] = np.nan
    
    temp_str[:, 0:(window//2)] = np.nan
    temp_str[:, (trialsinsess-window//2):trialsinsess] = np.nan

# take mean of smoothed data   
mean_unstr = (np.mean(np.array(temp_unstr), axis = 0))
mean_str = (np.mean(np.array(temp_str), axis = 0))

# plot line 1
plt.plot(np.arange(trialsinsess), mean_unstr, color1, label = label1) 
plt.fill_between(np.arange(trialsinsess), mean_unstr+sem(temp_unstr, nan_policy = 'omit'),
                 mean_unstr-sem(temp_unstr, nan_policy = 'omit'), color = color1, alpha = 0.2)

# plot line 2 (or comment)
plt.plot(np.arange(trialsinsess), mean_str, color2, label = label2)
plt.fill_between(np.arange(trialsinsess), mean_str+sem(temp_str, nan_policy = 'omit'), 
                 mean_str-sem(temp_str, nan_policy = 'omit'), color = color2, alpha = 0.2)

# plot significance
# pvals = ttest_rel(temp_str, temp_unstr)[1]
# pvals_ind = [ind for ind, i in enumerate(pvals) if i < 0.05]
# plt.scatter(pvals_ind, [0.65]*len(pvals_ind), marker = '.', color = 'r', alpha = 0.4) #(6, 2, 60) for asterisks

# plot settings
plt.xlabel('Trials in session', fontsize = 'x-large')
plt.ylabel('Performance index', fontsize = 'x-large')
plt.xlim(0, trialsinsess)
plt.axhline(0.625, color='k', linestyle = '--') # chance calc by
# plt.ylim(0.6, 0.85)
plt.xticks([0, 25, 50, 75, 100], [1, '', '', '', 100], color = 'grey', fontsize = 'large')
# plt.yticks([0.6, 0.65, 0.7, 0.75, 0.8, 0.85], ['0.60', '', '','','', 0.85], color = 'grey', fontsize = 'large')
# plt.ylim(0, 0.8)
plt.legend(loc='upper left')
sns.despine()
# sns.set_context('talk')
plt.tight_layout()
# plt.savefig('C:/Users/dlab/OneDrive - Indian Institute of Science/Drawings/awp1/entropy_n_21.png', dpi = 300)

IndexError: index 0 is out of bounds for axis 0 with size 0

# new rewprob-matched analysis

In [ ]:
results = []

# Process for each animal and offset in a single loop
for n in range(-20, 21):
    if n in [-2, -1, 0, 1, 2]:
        continue

    sessdf['entropy_shifted'] = (
        sessdf.groupby(['animal', 'session'])['entropy'].shift(-n)
    )

    ported = (
        sessdf[sessdf['task'] == 'unstr']
        .groupby(['animal', 'session', 'rewprob', 'reward'], as_index=False)['entropy_shifted']
        .mean()
    )

    grouped = ported.groupby(['animal', 'rewprob', 'reward'], as_index=False)['entropy_shifted'].mean()
    grouped['offset'] = n

    # Append to results
    results.append(grouped)

# Combine results
final_df = pd.concat(results, ignore_index=True)

In [1]:
rew_an_dict, unrew_an_dict, diff_dict = {}, {}, {}
# Add NaNs in the middle
nan_series = pd.Series([np.nan] * 5, index=range(-2, 3))

for an in final_df.animal.unique():
    # calc for each animal, for each rp
    for rewprob in [10, 15, 46, 80]:
        unrew = (final_df[(final_df['rewprob'] == rewprob) & (final_df['reward'] == 0) & (final_df['animal']==an)]
            .set_index('offset')['entropy_shifted'])
        rew = (final_df[(final_df['rewprob'] == rewprob) & (final_df['reward'] == 1) & (final_df['animal']==an)]
            .set_index('offset')['entropy_shifted'])
    
        rew_an_dict[(an, rewprob)] = pd.concat([rew.iloc[:18], nan_series, rew.iloc[18:]])
        unrew_an_dict[(an, rewprob)] = pd.concat([unrew.iloc[:18], nan_series, unrew.iloc[18:]])
        diff_dict[(an, rewprob)] = unrew_an_dict[(an, rewprob)] - rew_an_dict[(an, rewprob)]
    # if diff_dict[(an, rewprob)].isna().sum() > 0:
    #     print(an, rewprob)
data = pd.DataFrame(diff_dict).T.reset_index().groupby('level_1')[np.arange(-20, 21)].mean()

NameError: name 'pd' is not defined

In [255]:
colors = sns.color_palette("crest", as_cmap=True)(np.linspace(0, 1, 4))

for ind, rewprob in enumerate([10, 15, 46, 80]):
    plt.plot(data.T[ind], '.-', color = colors[ind], label = rewprob)
plt.legend()
sns.despine()
plt.tight_layout()

In [ ]:
# make a filter for sessions to have one of each rp in that session
# rpfull = sessdf.rewprobfull
# sessdf['num_rp'] = [len(np.unique(np.array(x.strip('[]').split(), dtype = int))) for x in rpfull] 

# mask = (
#     sessdf
#     .groupby(['animal', 'session'], as_index=False)
#     .apply(lambda g: g['rewprob'].nunique() == g['num_rp'].iloc[0])
# )
results = []
mask.columns = ['animal', 'session', 'is_valid']
cond = 'sham'
an_list = an_dict[cond]

filtered_sessdf = sessdf.merge(mask[mask['is_valid']], on=['animal', 'session'], how='inner')
filtered_sessdf = filtered_sessdf[(filtered_sessdf.animal.isin(an_list))]
# Process for each animal and offset in a single loop
for n in range(-20, 21):
    if n in [-2, -1, 0, 1, 2]:
        continue

    filtered_sessdf['entropy_shifted'] = (
        filtered_sessdf.groupby(['animal', 'session'])['entropy'].shift(-n)
    )

    ported = (
        filtered_sessdf[filtered_sessdf['task'] == 'unstr']
        .groupby(['animal', 'session', 'rewprob', 'reward'], as_index=False)['entropy_shifted']
        .mean()
    )

    grouped = ported.groupby(['animal', 'rewprob', 'reward'], as_index=False)['entropy_shifted'].mean()
    grouped['offset'] = n

    # Append to results
    results.append(grouped)

# Combine results
final_df = pd.concat(results, ignore_index=True)

rew_an_dict, unrew_an_dict, diff_dict = {}, {}, {}
# Add NaNs in the middle
nan_series = pd.Series([np.nan] * 5, index=range(-2, 3))

for an in final_df.animal.unique():
    # calc for each animal, for each rp
    for rewprob in [10, 15, 46, 80]:
        unrew = (final_df[(final_df['rewprob'] == rewprob) & (final_df['reward'] == 0) & (final_df['animal']==an)]
            .set_index('offset')['entropy_shifted'])
        rew = (final_df[(final_df['rewprob'] == rewprob) & (final_df['reward'] == 1) & (final_df['animal']==an)]
            .set_index('offset')['entropy_shifted'])
    
        rew_an_dict[(an, rewprob)] = pd.concat([rew.iloc[:18], nan_series, rew.iloc[18:]])
        unrew_an_dict[(an, rewprob)] = pd.concat([unrew.iloc[:18], nan_series, unrew.iloc[18:]])
        diff_dict[(an, rewprob)] = unrew_an_dict[(an, rewprob)] - rew_an_dict[(an, rewprob)]
data = pd.DataFrame(diff_dict).T.reset_index().groupby('level_1')[np.arange(-20, 21)].mean().T.to_numpy()

colors = sns.color_palette("crest", as_cmap=True)(np.linspace(0, 1, 4))

In [189]:
fig = plt.figure()
for ind, rewprob in enumerate([10, 15, 46, 80]):
    plt.plot(data.T[ind], '.-', color = colors[ind], label = rewprob)
plt.legend()
# plt.xticks(np.arange(-20, 20))
plt.xlabel('Trials')
plt.ylabel('Entropy no reward trials - reward trials')
plt.ylim(-0.1, 0.25)
sns.despine()
plt.tight_layout()

In [166]:
%matplotlib qt
plt.figure(figsize = (20, 10))
itera = 1
pn=1
col = 0
ax = plt.subplot(7,4,pn)
for key in diff_dict.keys():
    
    if itera % 4==0:
        ax = plt.subplot(7,4,pn)
        ax.set_title(key)
        pn+=1
        col = 0
    ax.plot(diff_dict[key], '.-', color = colors[col])
    col += 1
    itera +=1
sns.despine()
plt.tight_layout()

### why do DLS-lesioned animals only choose a few ports? are they able to perform perfectly (pre-lesion levels) in those sessions?

In [190]:
sessdf['high_port'] = sessdf['rewprobfull'].apply(lambda x: np.where(np.array(x.strip('[]').split())=='80')[0][0]+1)

In [191]:
plt.figure()
pn = 1
trialsinsess = 25
rr_port_dict = {}
for rat in sessdf[sessdf.task == 'dls'].animal.unique():
    ax = plt.subplot(3, 3, pn)
    raltzdf = sessdf[(sessdf.task == 'dls') & (sessdf.animal == rat) & (sessdf.shift_t0 == 1)]
    raltzdf = raltzdf.groupby('session').filter(lambda x: x.reward.size >= trialsinsess).groupby('session').head(trialsinsess)
    if raltzdf.empty==True:
        continue
    ax.plot(rr_mean_dict[(rat, 'unstr')], 'k', label = 'all')
    # ax.set_xlim(trialsinsess)
    # on this plot, plot mean RR curves for different highest rewprob along with transition matrix on right in smol
    # filter by sessions>= length
    for n in range(1,5):
        filtered = (sessdf[(sessdf.animal == rat) & (sessdf.high_port == n) & (sessdf.task == 'dls')]
                    .groupby('session')).filter(lambda x: x.reward.size >= trialsinsess)
        
        # take only those sessions
        filtered = filtered.groupby('session').head(trialsinsess)
        if filtered.empty==True:
            continue
            
        # convert to numpy array
        g = filtered.groupby('session').cumcount()
        L = np.array(filtered.set_index(['session', g])
               .unstack(fill_value=0)
               .stack().groupby(level=0)
               .apply(lambda x: x.rr.values.tolist())
               .tolist())
        rr_port_dict[n] = np.mean(L, axis = 0)
        ax.plot(rr_port_dict[n], label = n)
        ax.set_title(rat)
    ins = ax.inset_axes([0.7,0.7,0.25,0.25])
    
    sns.heatmap(pd.crosstab(raltzdf.port, raltzdf.choice_t1, normalize = 'index', dropna = False), 
                ax = ins,
                cmap = 'YlGnBu',
                annot = False,
                fmt = '.2f',
                vmin = 0.0,
                vmax = 0.7,
                square = True,
                mask = np.eye(4),
                xticklabels = np.arange(1,5),
                yticklabels = np.arange(1,5))
    ax.patch.set_facecolor('white')
    sns.despine()
    ax.legend()
    pn+=1

NameError: name 'rr_mean_dict' is not defined

# Early v/s late sessions - weekly analysis

In [71]:
# assign week numbers to each session based on the date
# min_year = pd.to_datetime(f"{sessdf['datetime'].astype('datetime64[ns]').dt.year.min()}-01")
# sessdf['week'] = (sessdf['datetime'].astype('datetime64[ns]') - min_year) // pd.to_timedelta('7D') + 1

# bin week numbers for each animal into 10 
# sessdf['week_bin'] = sessdf.groupby('animal')['week'].transform(lambda x: pd.qcut(x, 7, labels = False, duplicates='drop'))
# Mark each session number with a bin number. Each bin has 50 sessions.
bin_size = 50
sessdf['sess_bin'] = sessdf.groupby(['animal', 'task'])['session'].transform(lambda x: pd.cut(x, bins=range(0, x.max() + bin_size, bin_size), labels=False, right=False)+1)

In [565]:
#################################### regret ######################################
window = 7
trialsinsess = 100
fig = plt.figure(figsize=(20,10))

sessdf['regret'] = abs(sessdf['rewprob'] - 80)

reg_mean_dict_sess = {}

colors ={'str':'#408ec6', 'unstr':'#7a2048', 'dls':'xkcd:gold', 'dms':'xkcd:red', 'sham':'b', 'ds':'xkcd:grey',
         'dls_str':'red', 'dms_str':'xkcd:orange'} 

for ind, (animal, group) in enumerate(sessdf[sessdf.task == 'unstr'].groupby('animal')):
    ax = plt.subplot(5, 7, ind+1)
    cn = 0
    for sess_b in group.sess_bin.unique():

        # filter by sessions>= length
        filtered = (group[group.task.isin(['unstr']) & (group.sess_bin == sess_b)]
                    .groupby('session')).filter(lambda x: x.reward.size >= trialsinsess)

        # take only those sessions
        filtered = filtered.groupby('session').head(trialsinsess)
        
        if animal=='Nidorina':
            filtered = filtered[filtered.datetime.astype('datetime64[ns]')<datetime.datetime(2024, 5, 8)]

        if filtered.empty==True:
            continue

        # convert to numpy array
        g = filtered.groupby('session').cumcount()
        L = np.array(filtered.set_index(['session',g])
                .unstack(fill_value=0)
                .stack().groupby(level=0)
                .apply(lambda x: x.regret.values.tolist())
                .tolist())
        # map colors from cmap viridis to each session bin
        colors = plt.cm.cool_r(np.linspace(0, 1, len(group.sess_bin.unique())))

        # mean
        reg_mean = np.mean(L, axis = 0)
        reg_mean_dict_sess[animal, sess_b] = reg_mean
        ax.plot(reg_mean, color = colors[cn])
        cn+=1
    ax.set_title(animal)
    ax.set_xlabel('Trial')

sns.despine()
plt.tight_layout()

In [550]:
# plt.style.use('ggplot')
sns.set_style('ticks')
sns.set_context('notebook')

fig = plt.figure(figsize = (5,4.5))
smooth = True
# colors = plt.cm.cool_r(np.linspace(0, 1, len(sessdf.sess_bin.unique())+1))
colors = plt.cm.cool_r(np.linspace(0, 1, 8))

fish = ['test05022023', 'Blissey', 'Chikorita', 'Darkrai', 'Eevee']
c = [0, 3, 7]
ci = 0
for sess_bin in range(1, 9):
    temp_unstr = []
    for (animal, env) in reg_mean_dict_sess.keys():
        # if animal in fish:
        if env == sess_bin:
            temp_unstr.append(reg_mean_dict_sess[(animal, env)])

    # calculate performance index
    temp_unstr = (1-np.array(temp_unstr)/100)
    temp_unstr = np.array(temp_unstr)
    # temp_str = np.array(temp_str)

    # smoothing details
    if smooth == True:           
        for i, row in enumerate(temp_unstr):
            temp_unstr[i] = np.convolve(row, np.ones(window)/window, 'same')
        
        # works only for odd windows
        temp_unstr[:, 0:(window//2)] = np.nan
        temp_unstr[:, (trialsinsess-window//2):trialsinsess] = np.nan

    # temp_unstr = [temp_unstr[i] for i in [0, 2, 6]]


    # take mean of smoothed data   
    mean_unstr = (np.mean(np.array(temp_unstr), axis = 0))

    # plot line 1
    plt.plot(np.arange(trialsinsess), mean_unstr, color = colors[c[ci]], label = f'sess bin {sess_bin}') 
    plt.fill_between(np.arange(trialsinsess), mean_unstr+sem(temp_unstr, nan_policy = 'omit'),
                    mean_unstr-sem(temp_unstr, nan_policy = 'omit'), alpha = 0.2, color = colors[c[ci]])
    ci+=1
plt.xlabel('Trials in session', fontsize = 'x-large')
plt.ylabel('Performance index', fontsize = 'x-large')
plt.xlim(0, trialsinsess)
# plt.ylim(0.1, 0.6)
sns.despine()


plt.xticks([0, 50, 100], [1, '', 100], fontsize = 'large', color = 'grey')
plt.yticks([0.6, 0.7,0.8, 0.9], [0.6, '', '', 0.9], fontsize = 'large', color = 'grey')
plt.legend(['Naive','_fish_', 'Intermediate','_fish_', 'Expert'], loc = 'upper left')
plt.title(f'each session bin = {bin_size} sessions, n = {temp_unstr.shape[0]}')
plt.tight_layout()

C:\Users\dlab\AppData\Local\Temp\ipykernel_23560\916938773.py:54: UserWarning: The label '_fish_' of <matplotlib.collections.PolyCollection object at 0x000001C4AA1DE5F0> starts with '_'. It is thus excluded from the legend.
  plt.legend(['Naive','_fish_', 'Intermediate','_fish_', 'Expert'], loc = 'upper left')
C:\Users\dlab\AppData\Local\Temp\ipykernel_23560\916938773.py:54: UserWarning: The label '_fish_' of <matplotlib.collections.PolyCollection object at 0x000001C4AA1DEBF0> starts with '_'. It is thus excluded from the legend.
  plt.legend(['Naive','_fish_', 'Intermediate','_fish_', 'Expert'], loc = 'upper left')


In [458]:
# transition matrices for first n bins of sessions 
ind = 1
trialsinsess = 100
fig = plt.figure(figsize = (10, 4))
tempdf = sessdf.groupby(['animal','session']).filter(lambda x: x.reward.size >= trialsinsess).groupby(['animal','session']).head(trialsinsess)
tempdf = tempdf[~(tempdf.animal.isin(['Kakuna', 'Finneon']))]
an_list = tempdf.animal.unique()
mat = np.zeros((len(an_list), 4, 4))


for sess_bin in range(1,9):
    ax = plt.subplot(2, 4, sess_bin)
    temperdf = tempdf[(tempdf.sess_bin == sess_bin) & (tempdf.task == 'unstr')]
    sns.heatmap(pd.crosstab(temperdf.port, temperdf.choice_t1, normalize = 'index', dropna = False),
                cmap = parula_map,
                annot = False,
                fmt = '.2f',
                vmin = 0.0,
                vmax = 0.12,
                square = True,
                # mask = np.eye(4),
                xticklabels = np.arange(1,5),
                yticklabels = np.arange(1,5),
                ax = ax)
    ax.set_xlabel('')
    ax.set_ylabel('')
    ax.set_title(f'Sessions {sess_bin*bin_size-49} to {sess_bin*bin_size}')
fig.supxlabel('Choice at trial t+1')
fig.supylabel('Choice at trial t')
plt.tight_layout()

In [953]:
# difference transition matrices for first n bins of sessions 
ind = 1
trialsinsess = 100

fig = plt.figure(figsize = (10, 4))
tempdf = sessdf.groupby(['animal','session']).filter(lambda x: x.reward.size >= trialsinsess).groupby(['animal','session']).head(trialsinsess)
tempdf = tempdf[~(tempdf.animal.isin(['Kakuna', 'Finneon']))]
an_list = tempdf.animal.unique()
mat = np.zeros((len(an_list), 4, 4))


for sess_bin in range(1,9):
    ax = plt.subplot(2, 4, sess_bin)
    temperdf = tempdf[(tempdf.sess_bin == sess_bin) & (tempdf.task == 'unstr')]
    rew_mat = pd.crosstab(temperdf[temperdf.reward==1].port, temperdf[temperdf.reward==1].choice_t1, normalize = 'index', dropna = False)
    unrew_mat = pd.crosstab(temperdf[temperdf.reward==0].port, temperdf[temperdf.reward==0].choice_t1, normalize = 'index', dropna = False)
    sns.heatmap(rew_mat - unrew_mat,
                cmap = "seismic",
                annot = False,
                fmt = '.2f',
                vmin = -0.3,
                vmax = 0.3,
                square = True,
                # mask = np.eye(4),
                xticklabels = np.arange(1,5),
                yticklabels = np.arange(1,5),
                ax = ax)
    ax.set_xlabel('')
    ax.set_ylabel('')
    ax.set_title(f'Sessions {sess_bin*bin_size-49} to {sess_bin*bin_size}')
fig.supxlabel('Choice at trial t+1')
fig.supylabel('Choice at trial t')
plt.tight_layout()

c:\Users\dlab\miniforge3\envs\rishika\lib\site-packages\ipykernel\eventloops.py:145: UserWarning: The figure layout has changed to tight
  el.exec() if hasattr(el, "exec") else el.exec_()


In [569]:
# distance moved for first n bins of sessions 
ind = 1
trialsinsess = 100
fig = plt.figure(figsize = (9, 5))
tempdf = sessdf.groupby(['animal','session']).filter(lambda x: x.reward.size >= trialsinsess).groupby(['animal','session']).head(trialsinsess)
tempdf = tempdf[~(tempdf.animal.isin(['Kakuna', 'Finneon']))]
an_list = tempdf.animal.unique()
colors = plt.cm.cool_r(np.linspace(0, 1, 8))


for sess_bin in range(1,9):
    ax = plt.subplot(2, 4, sess_bin)
    temperdf = tempdf[(tempdf.sess_bin == sess_bin) & (tempdf.shift_t0 == 1)]
    # calculate for each animal, mean and sem
    x = (temperdf.groupby('animal', as_index = False)
                          .dist_abs
                          .value_counts(normalize = True))
    x.set_index('dist_abs', inplace = True)
    x['divisor']=pd.Series(randdisp).value_counts(normalize = True).sort_index()
    x['ratio']= x.proportion/x.divisor
    x['log_odds']= x.ratio.apply(np.log2)
    x['difference']=x.proportion-x.divisor
    
    mean_calc = x.reset_index().groupby('dist_abs').difference.mean()
    sem_calc = x.reset_index().groupby('dist_abs').difference.sem()

    # plot average statistic - although could have calculated this in a much much simpler way(?)
    # statmat = x.sort_values(['dist_abs', 'animal']).reset_index()[['dist_abs', 'difference']].to_numpy().reshape(3, -1, 2)
    # avg_grouped_statmat =  np.mean(statmat, axis = 1)

    # plot here
    ax.bar(np.array([1,2,3]), mean_calc, yerr = sem_calc, color = colors[sess_bin-1], width = 0.7)
    ax.set_xticks([1,2,3], ['1', '2', '3'], fontsize = 'large', color = 'grey')
    ax.axhline(0, color='k')
    ax.set_yticks([-0.2, 0, 0.2, 0.4], ['-0.2', '0.0', '0.2', '0.4'], fontsize = 'large', color = 'grey')
    sns.despine()
    ax.set_xlabel('')
    ax.set_ylabel('')
    ax.set_title(f'Sessions {sess_bin*bin_size-49} to {sess_bin*bin_size}')
fig.supxlabel('Distance')
fig.supylabel("$\Delta$ probability of switching")
plt.tight_layout()

AttributeError: 'DataFrameGroupBy' object has no attribute 'dist_abs'

In [586]:
trialsinsess = 100
tempdf = sessdf.groupby(['animal','session']).filter(lambda x: x.reward.size >= trialsinsess).groupby(['animal','session']).head(trialsinsess)
# & tempdf.animal.isin(['test05022023','Blissey', 'Chikorita', 'Darkrai', 'Eevee', 'Goldeen', 'Hoppip', 'Inkay', 'Jirachi', 'Kirlia', 'Nidorina', 'Phione', 'Quilava', 'Raltz', 'Togepi', 'Umbreon', 'Vulpix', 'Xatu', 'Yanma', 'Zacian'])
tempdf = tempdf[~(tempdf.animal.isin(['Kakuna', 'Finneon'])) & (tempdf.task == 'unstr')]
tempdf = tempdf[tempdf.sess_bin <= 8]

In [933]:
sns.displot(data = tempdf,
            x = 'port',
            stat = 'probability',
            hue = 'sess_bin',
            col = 'sess_bin',
            col_wrap = 8,
            common_norm = False,
            bins = 4,
            discrete = True, 
            palette = 'cool_r',
            kind = 'hist')
plt.xticks([1,2,3,4])

c:\Users\dlab\miniforge3\envs\rishika\lib\site-packages\seaborn\axisgrid.py:123: UserWarning: The figure layout has changed to tight
  self._figure.tight_layout(*args, **kwargs)


([<matplotlib.axis.XTick at 0x1c5470ccd30>,
 [Text(1, 0, '1'), Text(2, 0, '2'), Text(3, 0, '3'), Text(4, 0, '4')])

# Bias by port proportion

In [585]:
# get most sampled to least sampled port for each animal in each session bin (L to R)
tempdf = sessdf.groupby(['animal','session']).filter(lambda x: x.reward.size >= trialsinsess).groupby(['animal','session']).head(trialsinsess)
tempdf = tempdf[~(tempdf.animal.isin(['Kakuna', 'Finneon']))]

# take port proportion and arrange by highest to lowest. then average highest with highest, second highest with second highest etc
port_prop = tempdf[(tempdf.task == 'unstr') & (tempdf.sess_bin <= 8)].groupby(['animal', 'sess_bin'], as_index = False).port.value_counts(normalize = True, sort = True)
port_prop['highness'] = port_prop.groupby(['animal', 'sess_bin']).cumcount()+1
bars = port_prop.groupby(['sess_bin', 'highness']).proportion.mean().unstack().to_numpy()

err = port_prop.groupby(['sess_bin', 'highness']).proportion.sem().unstack().to_numpy()

colors = plt.cm.cool_r(np.linspace(0, 1, 8))
plt.figure(figsize = (17, 3.5))
for i in range(1, 9):
    ax = plt.subplot(1, 8, i)
    ax.bar(np.arange(1, 5), bars[i-1], color = colors[i-1], yerr = err[i-1])
    ax.set_title(f'sess bin {i}')
    ax.set_xticks([1, 4], ['Most \n sampled',  'Least \n sampled'], fontsize = 'large', color = 'grey')
    ax.set_ylim([0, 0.6])
    ax.set_yticks([0, 0.3, 0.6], ['0.0', '', '0.6'], fontsize = 'large', color = 'grey')
sns.despine()
plt.tight_layout()

In [590]:
# for each port, calculate the proportion of choices in each session bin. then calculate entropy in that session bin 
ys = port_prop.groupby(['animal', 'sess_bin']).proportion.apply(lambda x: entropy(x, base = 2)).groupby('sess_bin').mean().to_list()
# ys = [ys[0], ys[2], ys[7]]
yerrs = port_prop.groupby(['animal', 'sess_bin']).proportion.apply(lambda x: entropy(x, base = 2)).groupby('sess_bin').sem().to_list()
# yerrs = [yerrs[0], yerrs[2], yerrs[7]]
x = 1
colors = plt.cm.cool_r(np.linspace(0, 1, 8))
# colors = colors[[0, 3, 7]]
x = 1
for y, c in zip(ys, colors):
    plt.bar(x, y, color=c, yerr = yerrs[ys.index(y)])
    x +=1
plt.ylim(0.8, 1.6)
plt.yticks([0.8, 1.2, 1.6], [0.8, '', 1.6], fontsize = 'large', color = 'grey')
plt.xticks(np.arange(1, 9), np.arange(1, 9), fontsize = 'large', color = 'grey')
plt.xlabel('Session bin', fontsize = 'x-large')
plt.ylabel('Entropy (bits)', fontsize = 'x-large')
sns.despine()
plt.tight_layout()

In [806]:
# DLS group
# get most sampled to least sampled port for each animal in each session bin (L to R)
tempdf = sessdf.groupby(['animal','session']).filter(lambda x: x.reward.size >= trialsinsess).groupby(['animal','session']).head(trialsinsess)
tempdf = tempdf[~(tempdf.animal.isin(['Kakuna', 'Finneon']))]

def calc_bias(tempdf, mask):
# take port proportion and arrange by highest to lowest. then average highest with highest, second highest with second highest etc
    port_prop = tempdf[mask].groupby('animal', as_index = False).port.value_counts(normalize = True, sort = True)
    port_prop['highness'] = port_prop.groupby('animal').cumcount()+1
    y = port_prop.groupby(['animal', 'highness']).proportion.mean().unstack().T.apply(lambda x: entropy(x, base = 2))
    return y
fish = ['test05022023', 'Blissey', 'Chikorita', 'Darkrai', 'Eevee']


mask = (tempdf.task == 'unstr') & (tempdf.animal.isin(dls_an)) & (tempdf.sess_bin>=4)
y_pre = calc_bias(tempdf, mask)
y_pre_sem = y_pre.sem()

mask = (tempdf.task == 'dls') & (tempdf.animal.isin(dls_an))
y_les = calc_bias(tempdf, mask)
y_les_sem = y_les.sem()

plt.figure(figsize = (3.5, 4.5))
plt.bar([0, 1], [y_pre.mean(), y_les.mean()], yerr = [y_pre_sem, y_les_sem], color = ['xkcd:azure', 'xkcd:slate grey'], width = 0.6)
plt.xticks([0, 1], ['Pre-lesion', 'Post-lesion'], fontsize = 'large', color = 'grey')
plt.ylim(1.25, 2)
plt.yticks(fontsize = 'large', color = 'grey')
plt.title(round(ttest_rel(y_pre, y_les)[1], 4))
sns.despine()
plt.tight_layout()

In [810]:
plt.scatter(np.zeros(len(y_pre)), y_pre, color = 'xkcd:azure', alpha = 0.5)
plt.scatter(np.ones(len(y_les)), y_les, color = 'xkcd:slate grey', alpha = 0.5)
for row in y_pre.index:
    plt.plot([0, 1], [y_pre[row], y_les[row]], color = 'grey', alpha = 0.2)

plt.xticks([0, 1], ['Pre-lesion', 'Post-lesion'], fontsize = 'large', color = 'grey')
plt.ylim(0.0, 2)
plt.yticks(fontsize = 'large', color = 'grey')
plt.title(round(ttest_rel(y_pre, y_les)[1], 4))
sns.despine()
plt.tight_layout()

In [816]:
1- (tempdf[mask].groupby('animal').regret.mean()/100)

animal
Blissey    0.724974
Darkrai    0.728391
Inkay      0.835456
Mesprit    0.744715
Quilava    0.793589
Raltz      0.755387
Zacian     0.830155
Name: regret, dtype: float64

# Frac of sessions vs session length 

In [ ]:
plt.figure()
task_arr = ['unstr', 'str', 'dls', 'dms', 'sham']

an_list = [an_dict[task_arr[i]] for i in task_arr]
colors = ['xkcd:pumpkin orange', 'xkcd:emerald green', 'xkcd:red', 'xkcd:gold', 'grey']
for task in task_arr:
    lendf = sessdf[(sessdf.task == task) & (sessdf.animal.isin(an_list[task_arr.index(task)]))].groupby(['animal', 'session']).trial.nunique().reset_index()
    frac_array = np.zeros((2, lendf.trial.unique().shape[0]))
    frac_array[0] = lendf.trial.unique()
    frac_array[1] = [(lendf.trial>=trial).sum() for trial in lendf.trial.unique()]
    plt.scatter(frac_array[0], frac_array[1]/lendf.shape[0], s = 2, c = colors[task_arr.index(task)])
    if task =='unstr':
        plt.plot(np.arange(max(frac_array[0]))[::-1], np.arange(max(frac_array[0]))/max(frac_array[0]), color = 'k', lw = 0.2, label = "_nolegend_")

plt.ylabel('fraction of sessions with atleast x trials')
plt.xlabel('trials in session')
plt.legend(task_arr, markerscale = 5)
sns.despine()
plt.tight_layout()

In [512]:
plt.figure()
task_arr = ['unstr', 'sham']
fish = ['Blissey', 'Chikorita', 'Darkrai', 'Eevee', 'test05022023']
dls_an = ['Blissey', 'Darkrai', 'Inkay', 'Mesprit', 'Quilava', 'Raltz', 'Zacian']
dms_an = ['Hoppip', 'Kirlia', 'Nidorina', 'Togepi', 'Xatu']
sham_an = ['test05022023','Jirachi', 'Goldeen', 'Phione', 'Umbreon', 'Vulpix']
an_list = [sham_an, sham_an]
# colors = ['xkcd:pumpkin orange', 'xkcd:emerald green', 'xkcd:red', 'xkcd:gold', 'grey']
colors = ['xkcd:azure', 'xkcd:slate grey']
for task in task_arr:
    lendf = sessdf[(sessdf.task == task) & (sessdf.animal.isin(an_list[task_arr.index(task)]))].groupby(['animal', 'session']).trial.nunique().reset_index()
    if task == 'unstr':
        lendf = sessdf[(sessdf.task == task) & (sessdf.animal.isin(an_list[task_arr.index(task)]) & (sessdf.sess_bin >= 4))].groupby(['animal', 'session']).trial.nunique().reset_index()
    frac_array = np.zeros((2, lendf.trial.unique().shape[0]))
    frac_array[0] = lendf.trial.unique()
    indices = np.argsort(frac_array[0])
    frac_array[1] = [(lendf.trial>=trial).sum() for trial in lendf.trial.unique()]
    a1 = np.array([frac_array[0][i] for i in indices])
    b1 = np.array([frac_array[1][i] for i in indices])
    # plt.scatter(frac_array[0], frac_array[1]/lendf.shape[0], s = 2, color = colors[task_arr.index(task)])
    plt.plot(a1, b1/lendf.shape[0], '.-', color = colors[task_arr.index(task)])
    # if task =='unstr':
    #     plt.plot(np.arange(max(frac_array[0]))[::-1], np.arange(max(frac_array[0]))/max(frac_array[0]), color = 'k', lw = 0.2, label = "_nolegend_")

plt.ylabel('Fraction of sessions with atleast x trials', fontsize = 'x-large')
plt.xlabel('Trials in session', fontsize = 'x-large')
plt.xlim(0, 250)
plt.ylim(0, 1)
plt.yticks([0, 0.25, 0.5, 0.75, 1], ['0.00', '0.25', '0.50', '0.75', '1.00'], fontsize = 'large', color = 'grey')
plt.xticks([0, 50, 100, 150, 200, 250], ['0', '50', '100', '150', '200', '250'], fontsize = 'large', color = 'grey')
plt.legend(['Unstructured', 'Sham'], markerscale = 2)
sns.despine()
plt.tight_layout()

In [567]:
cols = ['level_0', 'level_1' ,90, 91, 92, 93, 94, 95, 96, 97, 98, 99]
perf_ind = pd.DataFrame(reg_mean_dict_sess).apply(lambda x: 1-x/100).T.reset_index()[cols]
perf_ind['avg'] = perf_ind[np.arange(90, 100)].mean(axis = 1)
ys = perf_ind.groupby('level_1').avg.mean().iloc[1:9].to_list()
yerrs = perf_ind.groupby('level_1').avg.sem().iloc[1:9].to_list()
x = 1
colors = plt.cm.cool_r(np.linspace(0, 1, len(ys)))
for y, c in zip(ys, colors):
    plt.bar(x, y, color=c, yerr = yerrs[ys.index(y)])
    x +=1
plt.ylim(0.7, 0.9)
plt.yticks([0.7, 0.8, 0.9], [0.7, '', 0.9], fontsize = 'large', color = 'grey')
plt.xticks(np.arange(1, 9), np.arange(1, 9), fontsize = 'large', color = 'grey')
plt.xlabel('Session bin', fontsize = 'x-large')
plt.ylabel('Performance index (trials 90-100)', fontsize = 'x-large')
sns.despine()
plt.tight_layout()

# Entropy in a trial across sessions

In [61]:
block_an_dict = {'unblocked': ['test05022023', 'Blissey', 'Chikorita', 'Darkrai', 'Eevee',
       'Goldeen', 'Hoppip', 'Inkay', 'Jirachi', 'Kirlia', 'Mesprit',
       'Nidorina', 'Oddish', 'Phione', 'Quilava', 'Raltz', 'Shinx',
       'Togepi', 'Umbreon', 'Vulpix', 'Xatu', 'Yanma', 'Zacian',
       'Alakazam', 'Bayleef', 'Cresselia'],
       'cued':['Emolga', 'Giratina','Haxorus', 'Ivysaur',
              'Jigglypuff', 'Lugia', 'Ninetales', 'Onix',
              'Pichu'],
       'uncued' : ['Quaxly', 'Torchic', 'Uxie', 'Vanillish',
              'Whismur', 'Xerneas', 'Yamper', 'Zorua']}

In [ ]:
#################################### moving entropy ######################################
# set_cwd('/home/rishika/sim/')
%matplotlib qt
window = 5
trialsinsess = 100
# fig = plt.figure(figsize=(20,10))
trialsinsess = 100
df = sessdf
from utils.dfLoading import subset_trials, add_block_groups, add_shift_info
subset_df_nohead = add_block_groups(df[(df.task == 'unstr') & (df.animal.isin(block_an_dict['cued']))])
subset_df_nohead = add_shift_info(subset_df_nohead)
subset_df_nohead = subset_trials(subset_df_nohead, trialsinsess=trialsinsess, head_trials=trialsinsess)


task = 'unstr'
block_2_exists = (subset_df_nohead[(subset_df_nohead.block_group>=2) & (subset_df_nohead.task == task)]
                  .groupby(['animal', 'sess_block'])
                  .size()
                  .reset_index()[['animal', 'sess_block']])

# remove animals with less than 10
block_2_exists = block_2_exists[block_2_exists.animal.isin(block_2_exists.animal.value_counts()[block_2_exists.animal.value_counts()>10].index)]

# find first blocks for these sessions
all_blocked_groups = subset_df_nohead[subset_df_nohead.task == task].merge(block_2_exists, on = ['animal', 'sess_block'])
first_blocks = all_blocked_groups[all_blocked_groups.block_group == 1]
second_blocks = all_blocked_groups[all_blocked_groups.block_group == 2]

other_blocks = all_blocked_groups[all_blocked_groups.block_group > 1]

entropy_mean_sess_dict = {}
i = 'unstr'
colors ={'str':'#408ec6', 'unstr':'#7a2048', 'dls':'xkcd:gold', 'dms':'xkcd:red', 'sham':'b', 'ds':'xkcd:orange'} 
for i in ['unstr', 'str', 'dms', 'dls', 'sham']:
    for ind, (animal, group) in enumerate(other_blocks.groupby('animal')):
        # ax = plt.subplot(5, 7, ind+1)

        if i == 'unstr':
            group = group[group.sess_bin>=4]
        
        # filter by sessions>= length
        filtered = (group[group.task.isin([i])]
                    .groupby('session')).filter(lambda x: x.reward.size >= trialsinsess)

        # take only those sessions
        filtered = filtered.groupby('session').head(trialsinsess)
        
        if animal=='Nidorina':
            filtered = filtered[filtered.datetime.astype('datetime64[ns]')<datetime.datetime(2024, 5, 8)]
        
        if filtered.empty==True:
            continue
            
        # convert to numpy array
        g = filtered.groupby('session').cumcount()
        L = np.array(filtered.set_index(['session',g])
                .unstack(fill_value=0)
                .stack(future_stack=True).groupby(level=0)
                .apply(lambda x: x.port.values.tolist())
                .tolist())
        # mean
        # entropy_trial = np.mean(L, axis = 0)
        entropy_trial = [entropy(calc_prob(L[:, col]), base = 2) for col in range(L.shape[1])]
        entropy_mean_sess_dict[animal, i] = entropy_trial
        # ax.plot(entropy_trial, label = i, c = colors[i])
        # ax.set_title(animal)
        # ax.legend()


# sns.despine()
# fig.supylabel('Entropy, averaged across sessions')
# plt.tight_layout()

c:\Users\dlab\banditAnalysis\utils\dfLoading.py:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['sess_block'] = 0
c:\Users\dlab\banditAnalysis\utils\dfLoading.py:63: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['sess_block'] = df.groupby(['animal', 'task']).sess_block.cumsum()
c:\Users\dlab\banditAnalysis\utils\dfLoading.py:65: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the d

In [94]:
# plot average reponse
# plt.figure(figsize = (5,4.5))
temp_unstr, temp_str = [], []
from scipy.stats import sem
# set properties
smooth = True
window = 3
label1, label2 = 'Pre-lesion', 'Post-lesion'
# label1, label2 = 'Unstructured', 'Structured'
color1, color2 = 'xkcd:cornflower', 'xkcd:slate grey'
# color1, color2 = 'xkcd:pumpkin orange', 'xkcd:emerald green'
# color1, color2 = '#7a2048', '#408ec6'

# set data for plotting
fish = ['Blissey', 'Darkrai', 'Mesprit', 'Quilava', 'Raltz', 'Inkay', 'Zacian']
# fish = ['Hoppip', 'Kirlia', 'Nidorina', 'Togepi', 'Xatu']
# fish = ['test05022023', 'Jirachi', 'Goldeen', 'Phione', 'Umbreon', 'Vulpix']
# fish = ['test05022023', 'Blissey', 'Chikorita', 'Darkrai', 'Eevee']
# fish = ['Alakazam', 'Yanma']
for (animal, env) in entropy_mean_sess_dict.keys():
    # if animal in fish:
    if env == 'unstr':
        temp_unstr.append(entropy_mean_sess_dict[(animal, env)])
        # elif (env == 'dls'):
        #     temp_str.append(entropy_mean_sess_dict[(animal, env)])
            
if smooth:           
    for i, row in enumerate(temp_unstr):
        temp_unstr[i] = np.convolve(row, np.ones(3)/3, 'same')
        # temp_str[i] = np.convolve(temp_str[i], np.ones(3)/3, 'same')
    #     temp_dms[i] = np.convolve(temp_dms[i], np.ones(3)/3, 'same')

    temp_unstr = np.array(temp_unstr)
    # temp_str = np.array(temp_str)

    # works only for odd windows
    temp_unstr[:, 0:(window//2)] = np.nan
    temp_unstr[:, (trialsinsess-window//2):trialsinsess] = np.nan
    
    # temp_str[:, 0:(window//2)] = np.nan
    # temp_str[:, (trialsinsess-window//2):trialsinsess] = np.nan
# # 

mean_unstr = (np.mean(np.array(temp_unstr), axis = 0))
# mean_str = (np.mean(np.array(temp_str), axis = 0))
# temp_str = (np.mean(np.array(temp_str), axis = 0))

# [plt.plot(np.arange(trialsinsess), x, color = 'grey', alpha = 0.1) for x in temp_unstr]
plt.plot(np.arange(trialsinsess), mean_unstr, color1, label = label1) #7a2048
plt.fill_between(np.arange(trialsinsess), mean_unstr+sem(temp_unstr, nan_policy = 'omit'),
                 mean_unstr-sem(temp_unstr, nan_policy = 'omit'), color = color1, alpha = 0.2)

# plt.plot(np.arange(trialsinsess), mean_str, color2, label = label2)

# plt.fill_between(np.arange(trialsinsess),mean_str+sem(temp_str, nan_policy = 'omit'), 
#                  mean_str-sem(temp_str, nan_policy = 'omit'), color = color2, alpha = 0.2)

plt.xlabel('Trials in session', fontsize = 'x-large')
plt.ylabel('Entropy (bits)', fontsize = 'x-large')
plt.xlim(0, trialsinsess)
# plt.ylim(1.5, 2.0)
plt.xticks([0, 25, 50, 75, 100], [1, '', '', '', 100], color = 'grey', fontsize = 'large')
# plt.yticks([1.25, 1.5, 1.75, 2.0], [1.25, '', '', 2.0], color = 'grey', fontsize = 'large')
plt.title(f'n = {temp_unstr.shape[0]}')
# plt.ylim(0, 0.8)
plt.legend()
sns.despine()
# sns.set_context('talk')
plt.tight_layout()

C:\Users\dlab\AppData\Local\Temp\ipykernel_27724\4229600451.py:50: SmallSampleWarning: After omitting NaNs, one or more axis-slices of one or more sample arguments is too small; corresponding elements of returned arrays will be NaN. See documentation for sample size requirements.
  plt.fill_between(np.arange(trialsinsess), mean_unstr+sem(temp_unstr, nan_policy = 'omit'),
C:\Users\dlab\AppData\Local\Temp\ipykernel_27724\4229600451.py:51: SmallSampleWarning: After omitting NaNs, one or more axis-slices of one or more sample arguments is too small; corresponding elements of returned arrays will be NaN. See documentation for sample size requirements.
  mean_unstr-sem(temp_unstr, nan_policy = 'omit'), color = color1, alpha = 0.2)


# switch distance comparisons

In [196]:
sns.displot(data = tempdf[(tempdf.task == 'unstr') & (tempdf.sess_bin <= 8)], x = 'disp', discrete = True, col = 'sess_bin', col_wrap = 4, hue = 'sess_bin', palette='cool_r', stat = 'probability', common_norm = False, alpha = 1)
plt.ylim(0., 0.08)

c:\Users\dlab\miniforge3\envs\rishika\lib\site-packages\seaborn\axisgrid.py:123: UserWarning: The figure layout has changed to tight
  self._figure.tight_layout(*args, **kwargs)


(0.0, 0.08)

In [23]:
tempdf = sessdf.groupby(['animal','session']).filter(lambda x: x.reward.size >= trialsinsess).groupby(['animal','session']).head(trialsinsess)
tempdf['trial_num'] = tempdf.groupby(['animal', 'session']).cumcount()
tempdf['trial_group'] = pd.cut(tempdf['trial_num']+1, bins = np.arange(0, 101, 10), labels = np.arange(1, 11))
sns.displot(data = tempdf[(tempdf.task == 'unstr') & (tempdf.sess_bin <= 8)], x = 'disp', discrete = True, col = 'trial_group', col_wrap = 5, hue = 'trial_group', palette='crest', stat = 'probability', common_norm = False, alpha = 1)
plt.ylim(0., 0.1)

(0.0, 0.1)

In [58]:
tempdf = sessdf.groupby(['animal','session']).filter(lambda x: x.reward.size >= trialsinsess).groupby(['animal','session']).head(trialsinsess)
tempdf = calc_dist_metric(tempdf, (tempdf.shift_t0==1) & (tempdf.task == 'unstr') & (tempdf.sess_bin <= 8))
y = tempdf.groupby(['sess_bin','animal']).d_value.mean().unstack().mean(axis = 1)
# y = [y[1], y[3], y[7]]
yerr = tempdf.groupby(['sess_bin','animal']).d_value.mean().unstack().sem(axis = 1)
# yerr = [yerr[1], yerr[3], yerr[7]]
x = tempdf.groupby(['sess_bin','animal']).d_chance.mean().unstack().mean(axis = 1)
# x = [x[1], x[3], x[7]]
# colors = [plt.cm.cool_r(np.linspace(0, 1, 8)[0]), plt.cm.cool_r(np.linspace(0, 1, 8)[3]), plt.cm.cool_r(np.linspace(0, 1, 8)[7])]
plt.figure()
for i in y.index[:8]:    
    plt.bar(i, y[i], color = plt.cm.cool_r(np.linspace(0, 1, 8)[int(i)-1]), yerr = yerr[i])
plt.bar(np.arange(len(x))+1, x, color = 'grey', alpha = 0.2)
# plt.xticks(np.arange(1, 4), ['Naive', 'Intermediate', 'Expert'], fontsize = 'large', color = 'grey')
plt.xticks(np.arange(1, 9), np.arange(1, 9), fontsize = 'large', color = 'grey')
plt.xlabel('Session bin', fontsize = 'x-large')
plt.ylim(1, 1.65)
plt.yticks([1, 1.2, 1.4, 1.6], [1,'', '', 1.6], fontsize = 'large', color = 'grey')
plt.ylabel('Distance metric', fontsize = 'x-large')
sns.despine()
plt.tight_layout()

C:\Users\dlab\AppData\Local\Temp\ipykernel_22440\59138584.py:11: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  d_chances = filtered.groupby(['animal', 'session']).apply(


In [19]:
tempdf = sessdf.groupby(['animal','session']).filter(lambda x: x.reward.size >= trialsinsess).groupby(['animal','session']).head(trialsinsess)
mask = ((tempdf.sess_bin >= 4) & (tempdf.shift_t0 == 1) & (tempdf.task == 'unstr'))
dist_moved = calc_dist_metric(tempdf, mask).groupby('animal').d_value.mean()
chance_move = calc_dist_metric(tempdf, mask).groupby('animal').d_chance.mean()

plt.figure(figsize=(3.5,4))
plt.bar(0, chance_move.mean(), color = 'grey',  yerr = chance_move.sem(), width = 0.6)
plt.bar(1, dist_moved.mean(), color = 'xkcd:cornflower', yerr = dist_moved.sem(), width = 0.6)
plt.xlim(-0.5, 1.5)
plt.ylim(1,1.65)
plt.xticks([0, 1], ['Chance', 'Animal data'], fontsize = 'large', color = 'grey')
plt.yticks([1, 1.2, 1.4, 1.6], [1,'', '', 1.6], fontsize = 'large', color = 'grey')
plt.ylabel('Distance moved', fontsize = 'x-large')
sns.despine()
plt.tight_layout()

C:\Users\dlab\AppData\Local\Temp\ipykernel_26796\2368743184.py:11: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  d_chances = filtered.groupby(['animal', 'session']).apply(
C:\Users\dlab\AppData\Local\Temp\ipykernel_26796\2368743184.py:11: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  d_chances = filtered.groupby(['animal', 'session']).apply(


In [40]:
# for r and nr
def calc_dist_metric(tempdf, mask):
    mean_distance = pd.Series([2, 4/3, 4/3, 2], index=[1, 2, 3, 4])

    filtered = tempdf[mask]

    # calculate `d_value` for each group
    d_values = filtered.groupby(['animal', 'session'])['disp'].apply(lambda x: x.abs().mean())

    # calculate `d_chance` for each group
    d_chances = filtered.groupby(['animal', 'session']).apply(
        lambda group: np.sum(group['port'].value_counts(normalize=True) * mean_distance)
    )

    tempdf['d_value'] = tempdf.set_index(['animal', 'session']).index.map(d_values).values
    tempdf['d_chance'] = tempdf.set_index(['animal', 'session']).index.map(d_chances).values
    return tempdf
trialsinsess = 100
tempdf = sessdf.groupby(['animal','session']).filter(lambda x: x.reward.size >= trialsinsess).groupby(['animal','session']).head(trialsinsess)
mask = (tempdf.reward == 0) & (tempdf.sess_bin >= 4) & (tempdf.shift_t0 == 1) & (tempdf.task == 'unstr')
nr = calc_dist_metric(tempdf, mask)
ys_nr = nr.groupby('animal').d_value.mean()

mask = (tempdf.reward == 1) & (tempdf.sess_bin >= 4) & (tempdf.shift_t0 == 1) & (tempdf.task == 'unstr')
r = calc_dist_metric(tempdf, mask)
ys_r = r.groupby('animal').d_value.mean()


C:\Users\dlab\AppData\Local\Temp\ipykernel_27724\2368743184.py:11: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  d_chances = filtered.groupby(['animal', 'session']).apply(
C:\Users\dlab\AppData\Local\Temp\ipykernel_27724\2368743184.py:11: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  d_chances = filtered.groupby(['animal', 'session']).apply(


In [42]:
plt.figure(figsize=(3.5,4))
plt.bar(1, nr.groupby('animal').d_chance.mean().mean(), color = 'grey', alpha = 0.2, width = 0.6)
plt.bar(1, ys_nr.mean(), color = 'blue', yerr = ys_nr.sem(), width = 0.6)
plt.hlines(y = nr.groupby('animal').d_chance.mean().mean(), xmin = -0.3, xmax = 0.3, color = 'grey', ls = '--', lw = 2)
plt.bar(0, r.groupby('animal').d_chance.mean().mean(), color = 'grey', alpha = 0.2, width = 0.6)
plt.bar(0, ys_r.mean(), color = 'red', yerr = ys_r.sem(), width = 0.6)
plt.hlines(y = r.groupby('animal').d_chance.mean().mean(), xmin = 1-0.3, xmax = 1+0.3, color = 'grey', ls = '--', lw = 2)

plt.xlim(-0.5, 1.5)
plt.ylim(1,1.7)
plt.xticks([0, 1], ['Reward', 'No reward'], fontsize = 'large', color = 'grey')
plt.yticks([1, 1.2, 1.4, 1.6], [1,'', '', 1.6], fontsize = 'large', color = 'grey')
plt.ylabel('Distance metric', fontsize = 'x-large')
sns.despine()
plt.tight_layout()

In [74]:
ttest_rel(ys_nr, ys_r, nan_policy='omit')

TtestResult(statistic=np.float64(5.6253652404618775), pvalue=np.float64(2.9110772608503093e-06), df=np.int64(33))

##  changes with trials

In [122]:
# select block 1 transitions with > 100 trials
plt.figure(figsize = (5,4.5))
colors = ['xkcd:cornflower', 'xkcd:slate grey']
labels = ['Pre-lesion', 'Post-lesion']
from utils.dfLoading import add_block_groups
for ind, task in enumerate(['unstr', 'sham']):
    df = add_block_groups(sessdf[sessdf.task == task])
    bl3 = (
        df[(df.block_group == 1) & (df.task == task)]
        .groupby(['animal', 'session'])
        .trial
        .count()[df[(df.block_group == 1) & (df.task == task)]
                    .groupby(['animal', 'session'])
                    .trial
                    .count()>100]
                    .index
                    .to_numpy()
                    )
    bl1_head = np.zeros(shape = (bl3.shape[0], 100))

    empty_row = []

    grouped = dict(tuple(df.groupby(['animal', 'session'])))
    for row, i in enumerate(bl3):
        key_bl3 = (i[0], i[1])
        key_bl2 = (i[0], i[1]-1)
        key_bl1 = (i[0], i[1]-2)
        if key_bl1 not in grouped or grouped[key_bl1].trial.count() < 100:
            empty_row.append(row)
            print(row)
        elif key_bl2 not in grouped or grouped[key_bl2].trial.count() < 100:
            empty_row.append(row)
            print(row)
        else:
            bl1_head[row] = grouped[key_bl1].head(100).port
    bl1_head[empty_row]= np.nan

    # each animal in switch_array
    an_ = np.array([val for i, val in enumerate(bl3) if i not in empty_row])[:, 0]
    switch_arr = np.abs(np.diff(bl1_head))
    switch_arr = switch_arr[~np.isnan(switch_arr).any(axis=1)]
    switch_arr[switch_arr == 0] = np.nan

    # keep rows without NaNs and align an_ with switch_arr
    mask = ~np.isnan(switch_arr).all(axis=1)
    switch_arr = switch_arr[mask]
    an_filtered = an_[mask]

    from collections import defaultdict
    grouped = defaultdict(list)
    for animal, row in zip(an_filtered, switch_arr):
        grouped[animal].append(row)

    # per-animal mean & sem (stack rows for each animal)
    group_mean = {k: np.nanmean(np.vstack(v), axis=0) for k, v in grouped.items()}

    # mean across animals (average of per-animal means)
    mean_across_animals = np.nanmean(np.vstack(list(group_mean.values())), axis=0)
    sem_across_animals = sem(np.vstack(list(group_mean.values())), axis=0, nan_policy='omit')

    smooth = True
    window = 3
    temp_unstr = np.ones(shape = (len(group_mean), 99))*np.nan
    trialsinsess = 99
    # smoothing details
    if smooth:           
        for i, row in enumerate(np.array(list(group_mean.values()))):
            temp_unstr[i] = np.convolve(row, np.ones(window)/window, 'same')
        
        # works only for odd windows
        temp_unstr[:, 0:(window//2)] = np.nan
        temp_unstr[:, (trialsinsess-window//2):trialsinsess] = np.nan

        mean_across_animals = np.nanmean(temp_unstr, axis = 0)
        sem_across_animals = sem(temp_unstr, axis = 0, nan_policy='omit')
    plt.plot(mean_across_animals, color=colors[ind], label = labels[ind])
    plt.fill_between(np.arange(mean_across_animals.size),
                 mean_across_animals - sem_across_animals,
                 mean_across_animals + sem_across_animals,
                 color=colors[ind], alpha=0.3)

# plot per-animal traces and the group average


plt.xlabel('Trials in session')
plt.xticks(np.arange(0, 101, 25), [1, '', '', '', 100])
plt.title(f'n = {len(group_mean)}')
plt.ylabel('Switch distance')
plt.legend(loc = 'upper right')
plt.tight_layout()

c:\Users\dlab\rishika_sim\utils\dfLoading.py:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['sess_block'] = 0
c:\Users\dlab\rishika_sim\utils\dfLoading.py:63: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['sess_block'] = df.groupby(['animal', 'task']).sess_block.cumsum()
c:\Users\dlab\rishika_sim\utils\dfLoading.py:65: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentat

0
1
2
11
12
18
19
26
27
28
29
30
31
32
33
34
36
37
38
39
40
92
93
103
105
106
110
112
114
115
118
124
125
128
129
132
136
137
138
139
141
142
143
144
145
146
148
149
150
151
152
154
155
156
157
158
163
164
165
166
168
169
170
171
172
173
174
178
179
180
184
185
191
192
194
195
198
208
211
222
229
235
238
245
255
257
259
271
272
273
274
282
284
291
304
307
308
316
333
337
338
339
341
354
360
364
371
374
375
376
379
386
392
394
395
403
412
420
424
426
428
430
436
451
452
458
459
466
467
468
471
472
473
474
475
476
485
486
488
489
500
501
511
516
517
518
519
522
523
532
533
554
555
573
574
584
585
594
595
598
603
607
608
609
610
617
618
649
650
736
737
805
806
882
883
933
934
964
965
966
967
968
969
970
971
972
988
989
991
992
1050
1051
1139
1140
1142
1143
1146
1147
1148
1151
1152
1154
1155
1156
1157
1158
1159
1160
1161
1162
1163
1164
1165
1166
1167
1170
1171
1172
1173
1174
1175
1176
1177
1178
1179
1180
1181
1182
1183
1184
1185
1186
1187
1188
1189
1190
1191
1193
1194
1195
1196
1197
1198
1

C:\Users\dlab\AppData\Local\Temp\ipykernel_8772\3843822824.py:55: RuntimeWarning: Mean of empty slice
  group_mean = {k: np.nanmean(np.vstack(v), axis=0) for k, v in grouped.items()}
C:\Users\dlab\AppData\Local\Temp\ipykernel_8772\3843822824.py:74: RuntimeWarning: Mean of empty slice
  mean_across_animals = np.nanmean(temp_unstr, axis = 0)
C:\Users\dlab\AppData\Local\Temp\ipykernel_8772\3843822824.py:75: SmallSampleWarning: After omitting NaNs, one or more axis-slices of one or more sample arguments is too small; corresponding elements of returned arrays will be NaN. See documentation for sample size requirements.
  sem_across_animals = sem(temp_unstr, axis = 0, nan_policy='omit')
c:\Users\dlab\rishika_sim\utils\dfLoading.py:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#retu

0
1
3
4
5
8
11
12
14
16
17
19
20
21
22
25
26
28
29
31
32
33
34
36
37
39
40
42
45
46
47
50
52
53
56
57
58
59
61
63
65
68
72
74
77
79
81
82
88
89
90
92
93
94
95
98
99
103
105
108
110
112
113
114
115
117
120
121
122
123
124
126
129
135
136
137
141
142
148
149
155
156
157
158
159
170
171
172
173
175
176
181
182
183
184
185
186
187
188
202
203
205
206
211
212
219
220
225
226
227
229
230
231
244
249
250
251
280
281
282
283
286
287
294
295
296
310
311
313
314
315
316
317
318
349
350
385
386
387
388
389
391
392
397
398
415
416
431
432
434
435
436
437
439
443
444
448
449
450
451
457
459
460
461
462
464
466
468
470
475
476
479
482
483
490
492
493
495
497
498
501
502
503
504
506
515
518
520
527
529
532
534
535
536
537
539
543
544
545
550
552
555
558
560
561
562
564
572
573
576
577
578
579
581
583
584
585
586
588
589
591
592
595
598
599
600
601
602
603
605
607
608
610
611
612
614
615
616
618
621
623
625
626
627
629
630
631
632
636
637
639
640
641
642
648
649
651
652
653
655
656
659
660
667
668
669

C:\Users\dlab\AppData\Local\Temp\ipykernel_8772\3843822824.py:55: RuntimeWarning: Mean of empty slice
  group_mean = {k: np.nanmean(np.vstack(v), axis=0) for k, v in grouped.items()}
C:\Users\dlab\AppData\Local\Temp\ipykernel_8772\3843822824.py:74: RuntimeWarning: Mean of empty slice
  mean_across_animals = np.nanmean(temp_unstr, axis = 0)
C:\Users\dlab\AppData\Local\Temp\ipykernel_8772\3843822824.py:75: SmallSampleWarning: After omitting NaNs, one or more axis-slices of one or more sample arguments is too small; corresponding elements of returned arrays will be NaN. See documentation for sample size requirements.
  sem_across_animals = sem(temp_unstr, axis = 0, nan_policy='omit')


## pairwise condition comparisons

In [82]:
# sessdf.drop(columns = ['d_value', 'd_chance'], inplace = True)
trialsinsess = 100
tempdf = sessdf.groupby(['animal','session']).filter(lambda x: x.reward.size >= trialsinsess).groupby(['animal','session']).head(trialsinsess)
struc = calc_dist_metric(tempdf, (tempdf.task == 'str') & (tempdf.animal.isin(fish)) & (tempdf.shift_t0 == 1))
tempdf = sessdf.groupby(['animal','session']).filter(lambda x: x.reward.size >= trialsinsess).groupby(['animal','session']).head(trialsinsess)
unstruc = calc_dist_metric(tempdf, (tempdf.task == 'unstr') & (tempdf.animal.isin(fish)) & (tempdf.shift_t0 == 1) & (tempdf.sess_bin >= 4))
y_str = struc[struc.animal.isin(fish)].groupby('animal').d_value.mean()
y_unstr = unstruc[unstruc.animal.isin(fish)].groupby('animal').d_value.mean()
plt.figure(figsize=(3.5,4))
plt.bar(0, unstruc[unstruc.animal.isin(fish)].groupby('animal').d_chance.mean().mean(), color = 'grey', alpha = 0.2, width = 0.6)
plt.bar(0, y_unstr.mean(), color = 'xkcd:pumpkin orange', yerr = y_unstr.sem(), width = 0.6)
plt.bar(1, struc[struc.animal.isin(fish)].groupby('animal').d_chance.mean().mean(), color = 'grey', alpha = 0.2, width = 0.6)
plt.bar(1, y_str.mean(), color = 'xkcd:emerald green', yerr = y_str.sem(), width = 0.6)
plt.xlim(-0.5, 1.5)
plt.ylim(1,1.7)
plt.xticks([0, 1], ['Unstructured', 'Structured'], fontsize = 'large', color = 'grey')
plt.yticks([1, 1.2, 1.4, 1.6], [1,'', '', 1.6], fontsize = 'large', color = 'grey')
plt.ylabel('Distance metric', fontsize = 'x-large')
sns.despine()
plt.tight_layout()
print(ttest_rel(y_unstr, y_str))

C:\Users\dlab\AppData\Local\Temp\ipykernel_8772\3827990971.py:11: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  d_chances = filtered.groupby(['animal', 'session']).apply(
C:\Users\dlab\AppData\Local\Temp\ipykernel_8772\3827990971.py:11: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  d_chances = filtered.groupby(['animal', 'session']).apply(


TtestResult(statistic=np.float64(0.13597937383112021), pvalue=np.float64(0.8984064333814432), df=np.int64(4))


In [43]:
tempdf = sessdf.groupby(['animal','session']).filter(lambda x: x.reward.size >= trialsinsess).groupby(['animal','session']).head(trialsinsess)
struc = calc_dist_metric(tempdf, (tempdf.task == 'dms') & (tempdf.animal.isin(dms_an)) & (tempdf.shift_t0 == 1))
tempdf = sessdf.groupby(['animal','session']).filter(lambda x: x.reward.size >= trialsinsess).groupby(['animal','session']).head(trialsinsess)
unstruc = calc_dist_metric(tempdf, (tempdf.task == 'unstr') & (tempdf.animal.isin(dms_an)) & (tempdf.shift_t0 == 1) & (tempdf.sess_bin >= 4))
y_str = struc[struc.animal.isin(dms_an)].groupby('animal').d_value.mean()
y_unstr = unstruc[unstruc.animal.isin(dms_an)].groupby('animal').d_value.mean()
width = 0.6
plt.figure(figsize=(3.5,4))
plt.bar(0, unstruc[unstruc.animal.isin(dms_an)].groupby('animal').d_chance.mean().mean(), color = 'grey', alpha = 0.2, width = 0.6)
plt.bar(0, y_unstr.mean(), color = 'xkcd:cornflower', yerr = y_unstr.sem(), width = 0.6)
plt.bar(1, struc[struc.animal.isin(dms_an)].groupby('animal').d_chance.mean().mean(), color = 'grey', alpha = 0.2, width = 0.6)
plt.bar(1, y_str.mean(), color = 'xkcd:slate grey', yerr = y_str.sem(), width = 0.6)
plt.xlim(-0.5, 1.5)
plt.ylim(1,1.9)
plt.xticks([0, 1], ['Pre-lesion', 'Post-lesion'], fontsize = 'large', color = 'grey')
plt.yticks([1, 1.2, 1.4, 1.6, 1.8], [1,'', '', '',1.8], fontsize = 'large', color = 'grey')
plt.ylabel('Distance metric', fontsize = 'x-large')
sns.despine()
plt.tight_layout()
print(ttest_rel(y_unstr, y_str))
print(ttest_rel(y_unstr, unstruc[unstruc.animal.isin(dms_an)].groupby('animal').d_chance.mean()))
print(ttest_rel(y_str, struc[struc.animal.isin(dms_an)].groupby('animal').d_chance.mean()))
# Statistical annotations using matplotlib
from scipy.stats import ttest_rel
annot = True
if annot:
    # Perform t-tests
    t_stat1, p_val1 = ttest_rel(y_unstr, y_str)
    t_stat2, p_val2 = ttest_rel(y_unstr, unstruc[unstruc.animal.isin(dms_an)].groupby('animal').d_chance.mean())
    t_stat3, p_val3 = ttest_rel(y_str, struc[struc.animal.isin(dms_an)].groupby('animal').d_chance.mean())

    # Add annotations
    max_y = (unstruc[unstruc.animal.isin(dms_an)].groupby('animal').d_chance.mean()).mean()

    if p_val1:
        plt.plot([0, 1], [max_y + 0.02, max_y + 0.02], color='black', linewidth=1)
        plt.text(0.5, max_y + 0.03, f'p={p_val1:.3f}', ha='center', fontsize=10)

    if p_val2:
        plt.plot([width/2 +0.05, width/2 + 0.05], [(y_unstr.mean()+1)/2, (max_y - y_unstr.mean())/2 + y_unstr.mean()], color='black', linewidth=1)
        plt.text(width/2 +0.1, y_unstr.mean() - 0.15, f'p={p_val2:.3f}', ha='center', fontsize=10, rotation = 270)

    if p_val3:
        plt.plot([width/2+1.05, width/2+1.05], [(y_str.mean()+1)/2, (max_y - y_str.mean())/2 + y_str.mean()], color='black', linewidth=1)
        plt.text(width/2+1.1, y_str.mean() - 0.15, f'p={p_val3:.3f}', ha='center', fontsize=10, rotation = 270)
plt.tight_layout()

C:\Users\dlab\AppData\Local\Temp\ipykernel_27724\2368743184.py:11: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  d_chances = filtered.groupby(['animal', 'session']).apply(
C:\Users\dlab\AppData\Local\Temp\ipykernel_27724\2368743184.py:11: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  d_chances = filtered.groupby(['animal', 'session']).apply(


TtestResult(statistic=np.float64(-0.9872314795757795), pvalue=np.float64(0.3447471510148764), df=np.int64(11))
TtestResult(statistic=np.float64(-2.9917392711353012), pvalue=np.float64(0.012259523813411255), df=np.int64(11))
TtestResult(statistic=np.float64(-2.5752125258030394), pvalue=np.float64(0.02580956474399872), df=np.int64(11))


In [45]:
tempdf = sessdf.groupby(['animal','session']).filter(lambda x: x.reward.size >= trialsinsess).groupby(['animal','session']).head(trialsinsess)
struc = calc_dist_metric(tempdf, (tempdf.task == 'dls') & (tempdf.animal.isin(dls_an)) & (tempdf.shift_t0 == 1))
tempdf = sessdf.groupby(['animal','session']).filter(lambda x: x.reward.size >= trialsinsess).groupby(['animal','session']).head(trialsinsess)
unstruc = calc_dist_metric(tempdf, (tempdf.task == 'unstr') & (tempdf.animal.isin(dls_an)) & (tempdf.shift_t0 == 1) & (tempdf.sess_bin >= 4))
y_str = struc[struc.animal.isin(dls_an)].groupby('animal').d_value.mean()
y_unstr = unstruc[unstruc.animal.isin(dls_an)].groupby('animal').d_value.mean()
width = 0.6
plt.figure(figsize=(3.5,4))
plt.bar(0, unstruc[unstruc.animal.isin(dls_an)].groupby('animal').d_chance.mean().mean(), color = 'grey', alpha = 0.2, width = 0.6)
plt.bar(0, y_unstr.mean(), color = 'xkcd:cornflower', yerr = y_unstr.sem(), width = 0.6)
plt.bar(1, struc[struc.animal.isin(dls_an)].groupby('animal').d_chance.mean().mean(), color = 'grey', alpha = 0.2, width = 0.6)
plt.bar(1, y_str.mean(), color = 'xkcd:slate grey', yerr = y_str.sem(), width = 0.6)
plt.xlim(-0.5, 1.5)
plt.ylim(1,1.9)
plt.xticks([0, 1], ['Pre-lesion', 'Post-lesion'], fontsize = 'large', color = 'grey')
plt.yticks([1, 1.2, 1.4, 1.6, 1.8], [1,'', '', '', 1.8], fontsize = 'large', color = 'grey')
plt.ylabel('Distance metric', fontsize = 'x-large')
sns.despine()
plt.tight_layout()
print(ttest_rel(y_unstr, y_str))
print(ttest_rel(y_unstr, unstruc[unstruc.animal.isin(dls_an)].groupby('animal').d_chance.mean()))
print(ttest_rel(y_str, struc[struc.animal.isin(dls_an)].groupby('animal').d_chance.mean()))
# Statistical annotations using matplotlib
from scipy.stats import ttest_rel
annot = True
if annot:
    # Perform t-tests
    t_stat1, p_val1 = ttest_rel(y_unstr, y_str)
    t_stat2, p_val2 = ttest_rel(y_unstr, unstruc[unstruc.animal.isin(dls_an)].groupby('animal').d_chance.mean())
    t_stat3, p_val3 = ttest_rel(y_str, struc[struc.animal.isin(dls_an)].groupby('animal').d_chance.mean())

    # Add annotations
    max_y = (unstruc[unstruc.animal.isin(dls_an)].groupby('animal').d_chance.mean()).mean()

    if p_val1:
        plt.plot([0, 1], [max_y + 0.02, max_y + 0.02], color='black', linewidth=1)
        plt.text(0.5, max_y + 0.03, f'p={p_val1:.3f}', ha='center', fontsize=10)

    if p_val2:
        plt.plot([width/2 +0.05, width/2 + 0.05], [(y_unstr.mean()+1)/2, (max_y - y_unstr.mean())/2 + y_unstr.mean()], color='black', linewidth=1)
        plt.text(width/2 +0.1, y_unstr.mean() - 0.15, f'p={p_val2:.3f}', ha='center', fontsize=10, rotation = 270)

    if p_val3:
        plt.plot([width/2+1.05, width/2+1.05], [(y_str.mean()+1)/2, (max_y - y_str.mean())/2 + y_str.mean()], color='black', linewidth=1)
        plt.text(width/2+1.1, y_str.mean() - 0.15, f'p={p_val3:.3f}', ha='center', fontsize=10, rotation = 270, color = 'red')
plt.tight_layout()

C:\Users\dlab\AppData\Local\Temp\ipykernel_27724\2368743184.py:11: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  d_chances = filtered.groupby(['animal', 'session']).apply(
C:\Users\dlab\AppData\Local\Temp\ipykernel_27724\2368743184.py:11: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  d_chances = filtered.groupby(['animal', 'session']).apply(


TtestResult(statistic=np.float64(-1.1594353333240175), pvalue=np.float64(0.268829526560833), df=np.int64(12))
TtestResult(statistic=np.float64(-5.5864437044468405), pvalue=np.float64(0.00011867162051950857), df=np.int64(12))
TtestResult(statistic=np.float64(-1.5816978693772266), pvalue=np.float64(0.1397034682278947), df=np.int64(12))


In [46]:
tempdf = sessdf.groupby(['animal','session']).filter(lambda x: x.reward.size >= trialsinsess).groupby(['animal','session']).head(trialsinsess)
struc = calc_dist_metric(tempdf, (tempdf.task == 'sham') & (tempdf.animal.isin(sham_an)) & (tempdf.shift_t0 == 1))
tempdf = sessdf.groupby(['animal','session']).filter(lambda x: x.reward.size >= trialsinsess).groupby(['animal','session']).head(trialsinsess)
unstruc = calc_dist_metric(tempdf, (tempdf.task == 'unstr') & (tempdf.animal.isin(sham_an)) & (tempdf.shift_t0 == 1) & (tempdf.sess_bin >= 4))
y_str = struc[struc.animal.isin(sham_an)].groupby('animal').d_value.mean()
y_unstr = unstruc[unstruc.animal.isin(sham_an)].groupby('animal').d_value.mean()
width = 0.6
plt.figure(figsize=(3.5,4))
plt.bar(0, unstruc.groupby('animal').d_chance.mean().mean(), color = 'grey', alpha = 0.2, width = 0.6)
plt.bar(0, y_unstr.mean(), color = 'xkcd:cornflower', yerr = y_unstr.sem(), width = 0.6)
plt.bar(1, struc.groupby('animal').d_chance.mean().mean(), color = 'grey', alpha = 0.2, width = 0.6)
plt.bar(1, y_str.mean(), color = 'xkcd:slate grey', yerr = y_str.sem(), width = 0.6)
plt.xlim(-0.5, 1.5)
plt.ylim(1,1.9)
plt.xticks([0, 1], ['Pre-lesion', 'Post-lesion'], fontsize = 'large', color = 'grey')
plt.yticks([1, 1.2, 1.4, 1.6, 1.8], [1,'', '', '', 1.8], fontsize = 'large', color = 'grey')
plt.ylabel('Distance metric', fontsize = 'x-large')
sns.despine()
plt.tight_layout()
print(ttest_rel(y_unstr, y_str))
print(ttest_rel(y_unstr, unstruc[unstruc.animal.isin(sham_an)].groupby('animal').d_chance.mean()))
print(ttest_rel(y_str, struc[struc.animal.isin(sham_an)].groupby('animal').d_chance.mean()))
# Statistical annotations using matplotlib
from scipy.stats import ttest_rel

# Perform t-tests
t_stat1, p_val1 = ttest_rel(y_unstr, y_str)
t_stat2, p_val2 = ttest_rel(y_unstr, unstruc[unstruc.animal.isin(sham_an)].groupby('animal').d_chance.mean())
t_stat3, p_val3 = ttest_rel(y_str, struc[struc.animal.isin(sham_an)].groupby('animal').d_chance.mean())

# Add annotations
max_y = (unstruc[unstruc.animal.isin(sham_an)].groupby('animal').d_chance.mean()).mean()

if p_val1:
    plt.plot([0, 1], [max_y + 0.02, max_y + 0.02], color='black', linewidth=1)
    plt.text(0.5, max_y + 0.03, f'p={p_val1:.3f}', ha='center', fontsize=10)

if p_val2:
    plt.plot([width/2 +0.05, width/2 + 0.05], [(y_unstr.mean()+1)/2, (max_y - y_unstr.mean())/2 + y_unstr.mean()], color='black', linewidth=1)
    plt.text(width/2 +0.1, y_unstr.mean() - 0.15, f'p={p_val2:.3f}', ha='center', fontsize=10, rotation = 270)

if p_val3:
    plt.plot([width/2+1.05, width/2+1.05], [(y_str.mean()+1)/2, (max_y - y_str.mean())/2 + y_str.mean()], color='black', linewidth=1)
    plt.text(width/2+1.1, y_str.mean() - 0.15, f'p={p_val3:.3f}', ha='center', fontsize=10, rotation = 270)
plt.tight_layout()

C:\Users\dlab\AppData\Local\Temp\ipykernel_27724\2368743184.py:11: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  d_chances = filtered.groupby(['animal', 'session']).apply(
C:\Users\dlab\AppData\Local\Temp\ipykernel_27724\2368743184.py:11: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  d_chances = filtered.groupby(['animal', 'session']).apply(


TtestResult(statistic=np.float64(-0.015446702498175658), pvalue=np.float64(0.9880540849631145), df=np.int64(8))
TtestResult(statistic=np.float64(-3.4781724611786644), pvalue=np.float64(0.008341986294539198), df=np.int64(8))
TtestResult(statistic=np.float64(-5.779804117773085), pvalue=np.float64(0.0004146098651563682), df=np.int64(8))


In [232]:
tempdf = sessdf.groupby(['animal','session']).filter(lambda x: x.reward.size >= trialsinsess).groupby(['animal','session']).head(trialsinsess)
struc = calc_dist_metric(tempdf, (tempdf.task == 'sham') & (tempdf.animal.isin(sham_an)) & (tempdf.shift_t0 == 1))
tempdf = sessdf.groupby(['animal','session']).filter(lambda x: x.reward.size >= trialsinsess).groupby(['animal','session']).head(trialsinsess)
unstruc = calc_dist_metric(tempdf, (tempdf.task == 'unstr') & (tempdf.animal.isin(sham_an)) & (tempdf.shift_t0 == 1) & (tempdf.sess_bin >= 4))
y_str = struc[struc.animal.isin(sham_an)].groupby('animal').d_value.mean()
y_unstr = unstruc[unstruc.animal.isin(sham_an)].groupby('animal').d_value.mean()
width = 0.6
plt.figure(figsize=(3.5,4))
plt.bar(0, unstruc.groupby('animal').d_chance.mean().mean(), color = 'grey', alpha = 0.2, width = 0.6)
plt.bar(0, y_unstr.mean(), color = 'xkcd:cornflower', yerr = y_unstr.sem(), width = 0.6)
plt.bar(1, struc.groupby('animal').d_chance.mean().mean(), color = 'grey', alpha = 0.2, width = 0.6)
plt.bar(1, y_str.mean(), color = 'xkcd:slate grey', yerr = y_str.sem(), width = 0.6)
plt.xlim(-0.5, 1.5)
plt.ylim(1,1.9)
plt.xticks([0, 1], ['Pre-lesion', 'Post-lesion'], fontsize = 'large', color = 'grey')
plt.yticks([1, 1.2, 1.4, 1.6, 1.8], [1,'', '', '', 1.8], fontsize = 'large', color = 'grey')
plt.ylabel('Distance metric', fontsize = 'x-large')
sns.despine()
plt.tight_layout()
print(ttest_rel(y_unstr, y_str))
print(ttest_rel(y_unstr, unstruc[unstruc.animal.isin(sham_an)].groupby('animal').d_chance.mean()))
print(ttest_rel(y_str, struc[struc.animal.isin(sham_an)].groupby('animal').d_chance.mean()))

# Statistical annotations using matplotlib
from scipy.stats import ttest_rel

# Perform t-tests
t_stat1, p_val1 = ttest_rel(y_unstr, y_str)
t_stat2, p_val2 = ttest_rel(y_unstr, unstruc[unstruc.animal.isin(sham_an)].groupby('animal').d_chance.mean())
t_stat3, p_val3 = ttest_rel(y_str, struc[struc.animal.isin(sham_an)].groupby('animal').d_chance.mean())

# Add annotations
max_y = (unstruc[unstruc.animal.isin(sham_an)].groupby('animal').d_chance.mean()).mean()

if p_val1:
    plt.plot([0, 1], [max_y + 0.02, max_y + 0.02], color='black', linewidth=1)
    plt.text(0.5, max_y + 0.03, f'p={p_val1:.3f}', ha='center', fontsize=10)

if p_val2:
    plt.plot([width/2 +0.05, width/2 + 0.05], [(y_unstr.mean()+1)/2, (max_y - y_unstr.mean())/2 + y_unstr.mean()], color='black', linewidth=1)
    plt.text(width/2 +0.1, y_unstr.mean() - 0.15, f'p={p_val2:.3f}', ha='center', fontsize=10, rotation = 270)

if p_val3:
    plt.plot([width/2+1.05, width/2+1.05], [(y_str.mean()+1)/2, (max_y - y_str.mean())/2 + y_str.mean()], color='black', linewidth=1)
    plt.text(width/2+1.1, y_str.mean() - 0.15, f'p={p_val3:.3f}', ha='center', fontsize=10, rotation = 270)
plt.tight_layout()

C:\Users\dlab\AppData\Local\Temp\ipykernel_22488\2368743184.py:11: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  d_chances = filtered.groupby(['animal', 'session']).apply(
C:\Users\dlab\AppData\Local\Temp\ipykernel_22488\2368743184.py:11: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  d_chances = filtered.groupby(['animal', 'session']).apply(


TtestResult(statistic=np.float64(-0.015446702498175658), pvalue=np.float64(0.9880540849631145), df=np.int64(8))
TtestResult(statistic=np.float64(-3.4781724611786644), pvalue=np.float64(0.008341986294539198), df=np.int64(8))
TtestResult(statistic=np.float64(-5.779804117773085), pvalue=np.float64(0.0004146098651563682), df=np.int64(8))


In [413]:
tempdf = sessdf.groupby(['animal','session']).filter(lambda x: x.reward.size >= trialsinsess).groupby(['animal','session']).head(trialsinsess)
unstruc = calc_dist_metric(tempdf, (tempdf.task == 'unstr') & (~tempdf.animal.isin(['Kakuna', "Finneon"])) & (tempdf.shift_t0 == 1) & (tempdf.sess_bin >= 4))
y_unstr = unstruc[~tempdf.animal.isin(['Kakuna', "Finneon"])].groupby('animal').d_value.mean()
plt.figure(figsize=(3.5,4))
plt.bar(0, unstruc.groupby('animal').d_chance.mean().mean(), color = 'grey', alpha = 0.2, width = 0.6)
plt.bar(1, y_unstr.mean(), color = 'xkcd:azure', yerr = y_unstr.sem(), width = 0.6)


<BarContainer object of 1 artists>

In [94]:
tempdf = sessdf.groupby(['animal','session']).filter(lambda x: x.reward.size >= trialsinsess).groupby(['animal','session']).head(trialsinsess)
les_r = calc_dist_metric(tempdf, (tempdf.task == 'sham') & (tempdf.animal.isin(sham_an)) & (tempdf.shift_t0 == 1) & (tempdf.reward == 1))

tempdf = sessdf.groupby(['animal','session']).filter(lambda x: x.reward.size >= trialsinsess).groupby(['animal','session']).head(trialsinsess)
les_nr = calc_dist_metric(tempdf, (tempdf.task == 'sham') & (tempdf.animal.isin(sham_an)) & (tempdf.shift_t0 == 1) & (tempdf.reward == 0))

tempdf = sessdf.groupby(['animal','session']).filter(lambda x: x.reward.size >= trialsinsess).groupby(['animal','session']).head(trialsinsess)
pre_r = calc_dist_metric(tempdf, (tempdf.task == 'unstr') & (tempdf.animal.isin(sham_an)) & (tempdf.shift_t0 == 1) & (tempdf.sess_bin >= 4) & (tempdf.reward == 1))

tempdf = sessdf.groupby(['animal','session']).filter(lambda x: x.reward.size >= trialsinsess).groupby(['animal','session']).head(trialsinsess)
pre_nr = calc_dist_metric(tempdf, (tempdf.task == 'unstr') & (tempdf.animal.isin(sham_an)) & (tempdf.shift_t0 == 1) & (tempdf.sess_bin >= 4) & (tempdf.reward == 0))

y_pre_r = pre_r[pre_r.animal.isin(sham_an)].groupby('animal').d_value.mean()
y_pre_nr = pre_nr[pre_nr.animal.isin(sham_an)].groupby('animal').d_value.mean()
y_pre_r_chance = pre_r[pre_r.animal.isin(sham_an)].groupby('animal').d_chance.mean()
y_pre_nr_chance = pre_nr[pre_nr.animal.isin(sham_an)].groupby('animal').d_chance.mean()

y_les_r = les_r[les_r.animal.isin(sham_an)].groupby('animal').d_value.mean()
y_les_nr = les_nr[les_nr.animal.isin(sham_an)].groupby('animal').d_value.mean()
y_les_r_chance = les_r[les_r.animal.isin(sham_an)].groupby('animal').d_chance.mean()
y_les_nr_chance = les_nr[les_nr.animal.isin(sham_an)].groupby('animal').d_chance.mean()

plt.figure(figsize=(3.5,4))
width = 0.4
# plt.bar(0-width/2, y_pre_r_chance.mean(), color = 'grey', alpha = 0.2, width = width)
# plt.bar(0+width/2, y_pre_nr_chance.mean(), color = 'grey', alpha = 0.2, width = width)

plt.bar(0-width/2, y_pre_r.mean(), color = 'red', yerr = y_pre_r.sem(), width = width)
plt.bar(0+width/2, y_pre_nr.mean(), color = 'blue', yerr = y_pre_nr.sem(), width = width)

plt.hlines(y_pre_r_chance.mean(), xmin = -width, xmax = 0, color = 'grey', ls = '--', lw = 2)
plt.hlines(y_pre_nr_chance.mean(), xmin = 0, xmax = width, color = 'grey', ls = '--', lw = 2)

# plt.bar(1-width/2, y_les_r_chance.mean(), color = 'grey', alpha = 0.2, width = width)
# plt.bar(1+width/2, y_les_nr_chance.mean(), color = 'grey', alpha = 0.2, width = width)

plt.bar(1-width/2, y_les_r.mean(), color = 'red', yerr = y_les_r.sem(), width = width)
plt.bar(1+width/2, y_les_nr.mean(), color = 'blue', yerr = y_les_nr.sem(), width = width)

plt.hlines(y_les_r_chance.mean(), xmin = 1-width, xmax = 1, color = 'grey', ls = '--', lw = 2)
plt.hlines(y_les_nr_chance.mean(), xmin = 1, xmax = width+1, color = 'grey', ls = '--', lw = 2)

plt.xlim(-0.5, 1.5)
plt.ylim(1,1.9)
plt.xticks([0, 1], ['Pre-lesion', 'Post-lesion'], fontsize = 'large', color = 'grey')
plt.yticks([1, 1.2, 1.4, 1.6, 1.8], [1,'', '', '', 1.8], fontsize = 'large', color = 'grey')
plt.ylabel('Distance metric', fontsize = 'x-large')
sns.despine()
plt.tight_layout()

C:\Users\dlab\AppData\Local\Temp\ipykernel_8772\3827990971.py:11: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  d_chances = filtered.groupby(['animal', 'session']).apply(
C:\Users\dlab\AppData\Local\Temp\ipykernel_8772\3827990971.py:11: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  d_chances = filtered.groupby(['animal', 'session']).apply(
C:\Users\dlab\AppData\Local\Temp\ipykernel_8772\3827990971.py:11: De

In [565]:
ttest_rel(y_pre_r, y_les_r), ttest_rel(y_pre_nr, y_les_nr)

(TtestResult(statistic=np.float64(-1.0516376713083069), pvalue=np.float64(0.33346442197301746), df=np.int64(6)),
 TtestResult(statistic=np.float64(-1.1102494601715123), pvalue=np.float64(0.30939408024261356), df=np.int64(6)))

In [91]:
tempdf = sessdf.groupby(['animal','session']).filter(lambda x: x.reward.size >= trialsinsess).groupby(['animal','session']).head(trialsinsess)
les_r = calc_dist_metric(tempdf, (tempdf.task == 'dls') & (tempdf.animal.isin(dls_an)) & (tempdf.shift_t0 == 1) & (tempdf.reward == 1))

tempdf = sessdf.groupby(['animal','session']).filter(lambda x: x.reward.size >= trialsinsess).groupby(['animal','session']).head(trialsinsess)
les_nr = calc_dist_metric(tempdf, (tempdf.task == 'dls') & (tempdf.animal.isin(dls_an)) & (tempdf.shift_t0 == 1) & (tempdf.reward == 0))

tempdf = sessdf.groupby(['animal','session']).filter(lambda x: x.reward.size >= trialsinsess).groupby(['animal','session']).head(trialsinsess)
pre_r = calc_dist_metric(tempdf, (tempdf.task == 'unstr') & (tempdf.animal.isin(dls_an)) & (tempdf.shift_t0 == 1) & (tempdf.sess_bin >= 4) & (tempdf.reward == 1))

tempdf = sessdf.groupby(['animal','session']).filter(lambda x: x.reward.size >= trialsinsess).groupby(['animal','session']).head(trialsinsess)
pre_nr = calc_dist_metric(tempdf, (tempdf.task == 'unstr') & (tempdf.animal.isin(dls_an)) & (tempdf.shift_t0 == 1) & (tempdf.sess_bin >= 4) & (tempdf.reward == 0))

y_pre_r = pre_r[pre_r.animal.isin(dls_an)].groupby('animal').d_value.mean()
y_pre_nr = pre_nr[pre_nr.animal.isin(dls_an)].groupby('animal').d_value.mean()
y_pre_r_chance = pre_r[pre_r.animal.isin(dls_an)].groupby('animal').d_chance.mean()
y_pre_nr_chance = pre_nr[pre_nr.animal.isin(dls_an)].groupby('animal').d_chance.mean()

y_les_r = les_r[les_r.animal.isin(dls_an)].groupby('animal').d_value.mean()
y_les_nr = les_nr[les_nr.animal.isin(dls_an)].groupby('animal').d_value.mean()
y_les_r_chance = les_r[les_r.animal.isin(dls_an)].groupby('animal').d_chance.mean()
y_les_nr_chance = les_nr[les_nr.animal.isin(dls_an)].groupby('animal').d_chance.mean()

plt.figure(figsize=(3.5,4))
width = 0.4
# plt.bar(0-width/2, y_pre_r_chance.mean(), color = 'grey', alpha = 0.2, width = width)
# plt.bar(0+width/2, y_pre_nr_chance.mean(), color = 'grey', alpha = 0.2, width = width)

plt.bar(0-width/2, y_pre_r.mean(), color = 'red', yerr = y_pre_r.sem(), width = width)
plt.bar(0+width/2, y_pre_nr.mean(), color = 'blue', yerr = y_pre_nr.sem(), width = width)

plt.hlines(y_pre_r_chance.mean(), xmin = -width, xmax = 0, color = 'grey', ls = '--', lw = 2)
plt.hlines(y_pre_nr_chance.mean(), xmin = 0, xmax = width, color = 'grey', ls = '--', lw = 2)

# plt.bar(1-width/2, y_les_r_chance.mean(), color = 'grey', alpha = 0.2, width = width)
# plt.bar(1+width/2, y_les_nr_chance.mean(), color = 'grey', alpha = 0.2, width = width)

plt.bar(1-width/2, y_les_r.mean(), color = 'red', yerr = y_les_r.sem(), width = width)
plt.bar(1+width/2, y_les_nr.mean(), color = 'blue', yerr = y_les_nr.sem(), width = width)

plt.hlines(y_les_r_chance.mean(), xmin = 1 - width, xmax = 1, color = 'grey', ls = '--', lw = 2)
plt.hlines(y_les_nr_chance.mean(), xmin = 1, xmax = 1+ width, color = 'grey', ls = '--', lw = 2)

plt.xlim(-0.5, 1.5)
plt.ylim(1,1.9)
plt.xticks([0, 1], ['Pre-lesion', 'Post-lesion'], fontsize = 'large', color = 'grey')
plt.yticks([1, 1.2, 1.4, 1.6, 1.8], [1,'', '', '', 1.8], fontsize = 'large', color = 'grey')
plt.ylabel('Distance metric', fontsize = 'x-large')
sns.despine()
plt.tight_layout()

C:\Users\dlab\AppData\Local\Temp\ipykernel_8772\3827990971.py:11: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  d_chances = filtered.groupby(['animal', 'session']).apply(
C:\Users\dlab\AppData\Local\Temp\ipykernel_8772\3827990971.py:11: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  d_chances = filtered.groupby(['animal', 'session']).apply(
C:\Users\dlab\AppData\Local\Temp\ipykernel_8772\3827990971.py:11: De

In [93]:
tempdf = sessdf.groupby(['animal','session']).filter(lambda x: x.reward.size >= trialsinsess).groupby(['animal','session']).head(trialsinsess)
les_r = calc_dist_metric(tempdf, (tempdf.task == 'dms') & (tempdf.animal.isin(dms_an)) & (tempdf.shift_t0 == 1) & (tempdf.reward == 1))

tempdf = sessdf.groupby(['animal','session']).filter(lambda x: x.reward.size >= trialsinsess).groupby(['animal','session']).head(trialsinsess)
les_nr = calc_dist_metric(tempdf, (tempdf.task == 'dms') & (tempdf.animal.isin(dms_an)) & (tempdf.shift_t0 == 1) & (tempdf.reward == 0))

tempdf = sessdf.groupby(['animal','session']).filter(lambda x: x.reward.size >= trialsinsess).groupby(['animal','session']).head(trialsinsess)
pre_r = calc_dist_metric(tempdf, (tempdf.task == 'unstr') & (tempdf.animal.isin(dms_an)) & (tempdf.shift_t0 == 1) & (tempdf.sess_bin >= 4) & (tempdf.reward == 1))

tempdf = sessdf.groupby(['animal','session']).filter(lambda x: x.reward.size >= trialsinsess).groupby(['animal','session']).head(trialsinsess)
pre_nr = calc_dist_metric(tempdf, (tempdf.task == 'unstr') & (tempdf.animal.isin(dms_an)) & (tempdf.shift_t0 == 1) & (tempdf.sess_bin >= 4) & (tempdf.reward == 0))

y_pre_r = pre_r[pre_r.animal.isin(dms_an)].groupby('animal').d_value.mean()
y_pre_nr = pre_nr[pre_nr.animal.isin(dms_an)].groupby('animal').d_value.mean()
y_pre_r_chance = pre_r[pre_r.animal.isin(dms_an)].groupby('animal').d_chance.mean()
y_pre_nr_chance = pre_nr[pre_nr.animal.isin(dms_an)].groupby('animal').d_chance.mean()

y_les_r = les_r[les_r.animal.isin(dms_an)].groupby('animal').d_value.mean()
y_les_nr = les_nr[les_nr.animal.isin(dms_an)].groupby('animal').d_value.mean()
y_les_r_chance = les_r[les_r.animal.isin(dms_an)].groupby('animal').d_chance.mean()
y_les_nr_chance = les_nr[les_nr.animal.isin(dms_an)].groupby('animal').d_chance.mean()

plt.figure(figsize=(3.5,4))
width = 0.4
# plt.bar(0-width/2, y_pre_r_chance.mean(), color = 'grey', alpha = 0.2, width = width)
# plt.bar(0+width/2, y_pre_nr_chance.mean(), color = 'grey', alpha = 0.2, width = width)

plt.bar(0-width/2, y_pre_r.mean(), color = 'red', yerr = y_pre_r.sem(), width = width)
plt.bar(0+width/2, y_pre_nr.mean(), color = 'blue', yerr = y_pre_nr.sem(), width = width)

plt.hlines(y_pre_r_chance.mean(), xmin = -width, xmax = 0, color = 'grey', ls = '--', lw = 2)
plt.hlines(y_pre_nr_chance.mean(), xmin = 0, xmax = width, color = 'grey', ls = '--', lw = 2)

# plt.bar(1-width/2, y_les_r_chance.mean(), color = 'grey', alpha = 0.2, width = width)
# plt.bar(1+width/2, y_les_nr_chance.mean(), color = 'grey', alpha = 0.2, width = width)

plt.bar(1-width/2, y_les_r.mean(), color = 'red', yerr = y_les_r.sem(), width = width)
plt.bar(1+width/2, y_les_nr.mean(), color = 'blue', yerr = y_les_nr.sem(), width = width)

plt.hlines(y_les_r_chance.mean(), xmin = 1-width, xmax = 1, color = 'grey', ls = '--', lw = 2)
plt.hlines(y_les_nr_chance.mean(), xmin = 1, xmax = 1+width, color = 'grey', ls = '--', lw = 2)

plt.xlim(-0.5, 1.5)
plt.ylim(1,1.9)
plt.xticks([0, 1], ['Pre-lesion', 'Post-lesion'], fontsize = 'large', color = 'grey')
plt.yticks([1, 1.2, 1.4, 1.6, 1.8], [1,'', '', '', 1.8], fontsize = 'large', color = 'grey')
plt.ylabel('Distance metric', fontsize = 'x-large')
sns.despine()
plt.tight_layout()

C:\Users\dlab\AppData\Local\Temp\ipykernel_8772\3827990971.py:11: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  d_chances = filtered.groupby(['animal', 'session']).apply(
C:\Users\dlab\AppData\Local\Temp\ipykernel_8772\3827990971.py:11: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  d_chances = filtered.groupby(['animal', 'session']).apply(
C:\Users\dlab\AppData\Local\Temp\ipykernel_8772\3827990971.py:11: De

In [420]:
tempdf = sessdf.groupby(['animal','session']).filter(lambda x: x.reward.size >= trialsinsess).groupby(['animal','session']).head(trialsinsess)
les_r = calc_dist_metric(tempdf, (tempdf.task == 'str') & (tempdf.animal.isin(fish)) & (tempdf.shift_t0 == 1) & (tempdf.reward == 1))

tempdf = sessdf.groupby(['animal','session']).filter(lambda x: x.reward.size >= trialsinsess).groupby(['animal','session']).head(trialsinsess)
les_nr = calc_dist_metric(tempdf, (tempdf.task == 'str') & (tempdf.animal.isin(fish)) & (tempdf.shift_t0 == 1) & (tempdf.reward == 0))

tempdf = sessdf.groupby(['animal','session']).filter(lambda x: x.reward.size >= trialsinsess).groupby(['animal','session']).head(trialsinsess)
pre_r = calc_dist_metric(tempdf, (tempdf.task == 'unstr') & (tempdf.animal.isin(fish)) & (tempdf.shift_t0 == 1) & (tempdf.sess_bin >= 4) & (tempdf.reward == 1))

tempdf = sessdf.groupby(['animal','session']).filter(lambda x: x.reward.size >= trialsinsess).groupby(['animal','session']).head(trialsinsess)
pre_nr = calc_dist_metric(tempdf, (tempdf.task == 'unstr') & (tempdf.animal.isin(fish)) & (tempdf.shift_t0 == 1) & (tempdf.sess_bin >= 4) & (tempdf.reward == 0))

y_pre_r = pre_r[pre_r.animal.isin(fish)].groupby('animal').d_value.mean()
y_pre_nr = pre_nr[pre_nr.animal.isin(fish)].groupby('animal').d_value.mean()
y_pre_r_chance = pre_r[pre_r.animal.isin(fish)].groupby('animal').d_chance.mean()
y_pre_nr_chance = pre_nr[pre_nr.animal.isin(fish)].groupby('animal').d_chance.mean()

y_les_r = les_r[les_r.animal.isin(fish)].groupby('animal').d_value.mean()
y_les_nr = les_nr[les_nr.animal.isin(fish)].groupby('animal').d_value.mean()
y_les_r_chance = les_r[les_r.animal.isin(fish)].groupby('animal').d_chance.mean()
y_les_nr_chance = les_nr[les_nr.animal.isin(fish)].groupby('animal').d_chance.mean()

plt.figure(figsize=(3.5,4))
width = 0.4
plt.bar(0-width/2, y_pre_r_chance.mean(), color = 'grey', alpha = 0.2, width = width)
plt.bar(0+width/2, y_pre_nr_chance.mean(), color = 'grey', alpha = 0.2, width = width)

plt.bar(0-width/2, y_pre_r.mean(), color = 'red', yerr = y_pre_r.sem(), width = width)
plt.bar(0+width/2, y_pre_nr.mean(), color = 'blue', yerr = y_pre_nr.sem(), width = width)

plt.bar(1-width/2, y_les_r_chance.mean(), color = 'grey', alpha = 0.2, width = width)
plt.bar(1+width/2, y_les_nr_chance.mean(), color = 'grey', alpha = 0.2, width = width)

plt.bar(1-width/2, y_les_r.mean(), color = 'red', yerr = y_les_r.sem(), width = width)
plt.bar(1+width/2, y_les_nr.mean(), color = 'blue', yerr = y_les_nr.sem(), width = width)

plt.xlim(-0.5, 1.5)
plt.ylim(1,1.9)
plt.xticks([0, 1], ['Unstructured', 'Structured'], fontsize = 'large', color = 'grey')
plt.yticks([1, 1.2, 1.4, 1.6, 1.8], [1,'', '', '', 1.8], fontsize = 'large', color = 'grey')
plt.ylabel('Distance metric', fontsize = 'x-large')
sns.despine()
plt.tight_layout()

# Switch probability pairwise comparison

In [ ]:
trialsinsess = 100
from utils.dfLoading import subset_trials
tempdf = subset_trials(sessdf, trialsinsess=trialsinsess, head_trials=True)
cond = 'str'
yr = tempdf[(tempdf.task == 'unstr') & (tempdf.animal.isin(an_dict[cond])) & (tempdf.sess_bin>=4)].groupby('animal').shift_t0.mean().mean()
ynr = tempdf[(tempdf.task == cond) & (tempdf.animal.isin(an_dict[cond]))].groupby('animal').shift_t0.mean().mean()

yr_err = tempdf[(tempdf.task == 'unstr') & (tempdf.animal.isin(an_dict[cond])) & (tempdf.sess_bin>=4)].groupby('animal').shift_t0.mean().sem()
ynr_err = tempdf[(tempdf.task == cond) & (tempdf.animal.isin(an_dict[cond]))].groupby('animal').shift_t0.mean().sem()

plt.figure(figsize=(3.5,4))
plt.bar(0, yr, color = 'xkcd:pumpkin orange', yerr = yr_err, width = 0.6)
plt.bar(1, ynr, color = 'xkcd:emerald green', yerr = ynr_err, width = 0.6)
plt.xticks([0, 1], ['Unstructured', 'Structured'], fontsize = 'large', color = 'grey')
plt.ylabel('Switch probability', fontsize = 'x-large')
# plt.yticks([0, 0.25], ['0.00', 0.25], fontsize = 'large', color = 'grey')///////////

sns.despine()
plt.tight_layout()

In [34]:
# from utils.dfLoading import subset_trials, add_shift_info
# sessdf = add_shift_info(sessdf)
# tempdf = subset_trials(sessdf, trialsinsess=trialsinsess, head_trials=trialsinsess)
yr = tempdf[(tempdf.task == 'unstr') & (tempdf.reward == 1) & (tempdf.sess_bin>=4)].groupby('animal').shift_t0.mean().mean()
ynr = tempdf[(tempdf.task == 'unstr') & (tempdf.reward == 0) & (tempdf.sess_bin>=4)].groupby('animal').shift_t0.mean().mean()

yr_err = tempdf[(tempdf.task == 'unstr') & (tempdf.reward == 1) & (tempdf.sess_bin>=4)].groupby('animal').shift_t0.mean().sem()
ynr_err = tempdf[(tempdf.task == 'unstr') & (tempdf.reward == 0) & (tempdf.sess_bin>=4)].groupby('animal').shift_t0.mean().sem()

plt.figure(figsize=(3.5,4))
plt.bar(0, yr, color = 'red', yerr = yr_err, width = 0.6)
plt.bar(1, ynr, color = 'blue', yerr = ynr_err, width = 0.6)
plt.xticks([0, 1], ['Reward', 'No reward'], fontsize = 'large', color = 'grey')
plt.ylabel('Switch probability', fontsize = 'x-large')
plt.yticks([0, 0.25], ['0.00', 0.25], fontsize = 'large', color = 'grey')
sns.despine()
plt.tight_layout()

In [105]:
fish = an_dict['str']
yr_unstr = tempdf[(tempdf.task == 'unstr') & (tempdf.reward == 1) & (tempdf.sess_bin>=4) & tempdf.animal.isin(fish)].groupby('animal').shift_t0.mean().mean()
ynr_unstr = tempdf[(tempdf.task == 'unstr') & (tempdf.reward == 0) & (tempdf.sess_bin>=4) & tempdf.animal.isin(fish)].groupby('animal').shift_t0.mean().mean()

yr_unstr_err = tempdf[(tempdf.task == 'unstr') & (tempdf.reward == 1) & (tempdf.sess_bin>=4) & tempdf.animal.isin(fish)].groupby('animal').shift_t0.mean().sem()
ynr_unstr_err = tempdf[(tempdf.task == 'unstr') & (tempdf.reward == 0) & (tempdf.sess_bin>=4) & tempdf.animal.isin(fish)].groupby('animal').shift_t0.mean().sem()

yr_str = tempdf[(tempdf.task == 'str') & (tempdf.reward == 1) & tempdf.animal.isin(fish)].groupby('animal').shift_t0.mean().mean()
ynr_str = tempdf[(tempdf.task == 'str') & (tempdf.reward == 0) & tempdf.animal.isin(fish)].groupby('animal').shift_t0.mean().mean()

yr_str_err = tempdf[(tempdf.task == 'str') & (tempdf.reward == 1) & tempdf.animal.isin(fish)].groupby('animal').shift_t0.mean().sem()
ynr_str_err = tempdf[(tempdf.task == 'str') & (tempdf.reward == 0) & tempdf.animal.isin(fish)].groupby('animal').shift_t0.mean().sem()

plt.figure(figsize=(3.5,4))
width = 0.4
plt.bar(0 - width/2, yr_unstr, color = 'red', yerr = yr_unstr_err, width = width)
plt.bar(0 + width/2, ynr_unstr, color = 'blue', yerr = yr_unstr_err, width = width)

plt.bar(1 - width/2, yr_str, color = 'red', yerr = yr_str_err, width = width)
plt.bar(1 + width/2, ynr_str, color = 'blue', yerr = ynr_str_err, width = width)
plt.ylim(0, 0.4)
plt.xticks([0, 1], ['Unstructured', 'Structured'], fontsize = 'large', color = 'grey')
plt.ylabel('Switch rate', fontsize = 'x-large')
plt.yticks([0, 0.2, 0.4], ['0.0', 0.2, 0.4], fontsize = 'large', color = 'grey')
sns.despine()
plt.tight_layout()

In [38]:
dms_an = an_dict['dms']
yr_unstr = tempdf[(tempdf.task == 'unstr') & (tempdf.reward == 1) & (tempdf.sess_bin>=4) & tempdf.animal.isin(dms_an)].groupby('animal').shift_t0.mean().mean()
ynr_unstr = tempdf[(tempdf.task == 'unstr') & (tempdf.reward == 0) & (tempdf.sess_bin>=4) & tempdf.animal.isin(dms_an)].groupby('animal').shift_t0.mean().mean()

yr_unstr_err = tempdf[(tempdf.task == 'unstr') & (tempdf.reward == 1) & (tempdf.sess_bin>=4) & tempdf.animal.isin(dms_an)].groupby('animal').shift_t0.mean().sem()
ynr_unstr_err = tempdf[(tempdf.task == 'unstr') & (tempdf.reward == 0) & (tempdf.sess_bin>=4)& tempdf.animal.isin(dms_an)].groupby('animal').shift_t0.mean().sem()

yr_str = tempdf[(tempdf.task == 'dms') & (tempdf.reward == 1) & tempdf.animal.isin(dms_an)].groupby('animal').shift_t0.mean().mean()
ynr_str = tempdf[(tempdf.task == 'dms') & (tempdf.reward == 0) & tempdf.animal.isin(dms_an)].groupby('animal').shift_t0.mean().mean()

yr_str_err = tempdf[(tempdf.task == 'dms') & (tempdf.reward == 1) & tempdf.animal.isin(dms_an)].groupby('animal').shift_t0.mean().sem()
ynr_str_err = tempdf[(tempdf.task == 'dms') & (tempdf.reward == 0) & tempdf.animal.isin(dms_an)].groupby('animal').shift_t0.mean().sem()

plt.figure(figsize=(3.5,4))
width = 0.4
plt.bar(0 - width/2, yr_unstr, color = 'red', yerr = yr_unstr_err, width = width)
plt.bar(0 + width/2, ynr_unstr, color = 'blue', yerr = yr_unstr_err, width = width)

plt.bar(1 - width/2, yr_str, color = 'red', yerr = yr_str_err, width = width)
plt.bar(1 + width/2, ynr_str, color = 'blue', yerr = ynr_str_err, width = width)
plt.ylim(0, 0.45)
plt.xticks([0, 1], ['Pre-lesion', 'Post-lesion'], fontsize = 'large', color = 'grey')
plt.ylabel('Switch rate', fontsize = 'x-large')
plt.yticks([0, 0.2, 0.4], ['0.0', 0.2, 0.4], fontsize = 'large', color = 'grey')
sns.despine()

# Statistical annotations
from scipy.stats import ttest_rel

# Rewarded pre vs post
rewarded_pre = tempdf[(tempdf.task == 'unstr') & (tempdf.reward == 1) & (tempdf.sess_bin>=4) & tempdf.animal.isin(dms_an)].groupby('animal').shift_t0.mean()
rewarded_post = tempdf[(tempdf.task == 'dms') & (tempdf.reward == 1) & tempdf.animal.isin(dms_an)].groupby('animal').shift_t0.mean()
t_stat, p_val = ttest_rel(rewarded_pre, rewarded_post)

# Unrewarded pre vs post
unrewarded_pre = tempdf[(tempdf.task == 'unstr') & (tempdf.reward == 0) & (tempdf.sess_bin>=4) & tempdf.animal.isin(dms_an)].groupby('animal').shift_t0.mean()
unrewarded_post = tempdf[(tempdf.task == 'dms') & (tempdf.reward == 0) & tempdf.animal.isin(dms_an)].groupby('animal').shift_t0.mean()
t_stat2, p_val2 = ttest_rel(unrewarded_pre, unrewarded_post)

# Add annotations
max_nr = max(ynr_unstr + ynr_unstr_err, ynr_str + ynr_str_err)

if p_val < 0.05:
    plt.plot([0 - width/2, 1 - width/2], [max_nr + 0.05, max_nr + 0.05], color='black', linewidth=1)
    plt.text(0.5, max_nr + 0.055, f'p={p_val:.3f}', ha='center', fontsize=10)
else:
    plt.plot([0 - width/2, 1 - width/2], [max_nr + 0.05, max_nr + 0.05], color='black', linewidth=1)
    plt.text(0.5, max_nr + 0.055, 'ns', ha='center', fontsize=10)

if p_val2 < 0.05:
    plt.plot([0 + width/2, 1 + width/2], [max_nr + 0.1, max_nr + 0.1], color='black', linewidth=1)
    plt.text(0.5, max_nr + 0.105, f'p={p_val2:.3f}', ha='center', fontsize=10)
else:
    plt.plot([0 + width/2, 1 + width/2], [max_nr + 0.1, max_nr + 0.1], color='black', linewidth=1)
    plt.text(0.5, max_nr + 0.105, 'ns', ha='center', fontsize=10)

plt.tight_layout()

In [37]:
dls_an = an_dict['dls']
yr_unstr = tempdf[(tempdf.task == 'unstr') & (tempdf.reward == 1) & (tempdf.sess_bin>=4) & tempdf.animal.isin(dls_an)].groupby('animal').shift_t0.mean().mean()
ynr_unstr = tempdf[(tempdf.task == 'unstr') & (tempdf.reward == 0) & (tempdf.sess_bin>=4) & tempdf.animal.isin(dls_an)].groupby('animal').shift_t0.mean().mean()

yr_unstr_err = tempdf[(tempdf.task == 'unstr') & (tempdf.reward == 1) & (tempdf.sess_bin>=4) & tempdf.animal.isin(dls_an)].groupby('animal').shift_t0.mean().sem()
ynr_unstr_err = tempdf[(tempdf.task == 'unstr') & (tempdf.reward == 0) & (tempdf.sess_bin>=4)& tempdf.animal.isin(dls_an)].groupby('animal').shift_t0.mean().sem()

yr_str = tempdf[(tempdf.task == 'dls') & (tempdf.reward == 1) & tempdf.animal.isin(dls_an)].groupby('animal').shift_t0.mean().mean()
ynr_str = tempdf[(tempdf.task == 'dls') & (tempdf.reward == 0) & tempdf.animal.isin(dls_an)].groupby('animal').shift_t0.mean().mean()

yr_str_err = tempdf[(tempdf.task == 'dls') & (tempdf.reward == 1) & tempdf.animal.isin(dls_an)].groupby('animal').shift_t0.mean().sem()
ynr_str_err = tempdf[(tempdf.task == 'dls') & (tempdf.reward == 0) & tempdf.animal.isin(dls_an)].groupby('animal').shift_t0.mean().sem()

plt.figure(figsize=(3.5,4))
width = 0.4
plt.bar(0 - width/2, yr_unstr, color = 'red', yerr = yr_unstr_err, width = width)
plt.bar(0 + width/2, ynr_unstr, color = 'blue', yerr = yr_unstr_err, width = width)

plt.bar(1 - width/2, yr_str, color = 'red', yerr = yr_str_err, width = width)
plt.bar(1 + width/2, ynr_str, color = 'blue', yerr = ynr_str_err, width = width)
plt.ylim(0, 0.45)
plt.xticks([0, 1], ['Pre-lesion', 'Post-lesion'], fontsize = 'large', color = 'grey')
plt.ylabel('Switch rate', fontsize = 'x-large')
plt.yticks([0, 0.2, 0.4], ['0.0', 0.2, 0.4], fontsize = 'large', color = 'grey')
sns.despine()
plt.tight_layout()
# Statistical annotations
from scipy.stats import ttest_rel

# Rewarded pre vs post
rewarded_pre = tempdf[(tempdf.task == 'unstr') & (tempdf.reward == 1) & (tempdf.sess_bin>=4) & tempdf.animal.isin(dls_an)].groupby('animal').shift_t0.mean()
rewarded_post = tempdf[(tempdf.task == 'dls') & (tempdf.reward == 1) & tempdf.animal.isin(dls_an)].groupby('animal').shift_t0.mean()
t_stat, p_val = ttest_rel(rewarded_pre, rewarded_post)

# Unrewarded pre vs post
unrewarded_pre = tempdf[(tempdf.task == 'unstr') & (tempdf.reward == 0) & (tempdf.sess_bin>=4) & tempdf.animal.isin(dls_an)].groupby('animal').shift_t0.mean()
unrewarded_post = tempdf[(tempdf.task == 'dls') & (tempdf.reward == 0) & tempdf.animal.isin(dls_an)].groupby('animal').shift_t0.mean()
t_stat2, p_val2 = ttest_rel(unrewarded_pre, unrewarded_post)

# Add annotations
max_nr = max(ynr_unstr + ynr_unstr_err, ynr_str + ynr_str_err)

if p_val < 0.05:
    plt.plot([0 - width/2, 1 - width/2], [max_nr + 0.05, max_nr + 0.05], color='black', linewidth=1)
    plt.text(0.5, max_nr + 0.055, f'p={p_val:.3f}', ha='center', fontsize=10)
else:
    plt.plot([0 - width/2, 1 - width/2], [max_nr + 0.05, max_nr + 0.05], color='black', linewidth=1)
    plt.text(0.5, max_nr + 0.055, 'ns', ha='center', fontsize=10)

if p_val2 < 0.05:
    plt.plot([0 + width/2, 1 + width/2], [max_nr + 0.1, max_nr + 0.1], color='black', linewidth=1)
    plt.text(0.5, max_nr + 0.105, f'p={p_val2:.3f}', ha='center', fontsize=10)
else:
    plt.plot([0 + width/2, 1 + width/2], [max_nr + 0.1, max_nr + 0.1], color='black', linewidth=1)
    plt.text(0.5, max_nr + 0.105, 'ns', ha='center', fontsize=10)

In [39]:
sham_an = an_dict['sham']
yr_unstr = tempdf[(tempdf.task == 'unstr') & (tempdf.reward == 1) & (tempdf.sess_bin>=4) & tempdf.animal.isin(sham_an)].groupby('animal').shift_t0.mean().mean()
ynr_unstr = tempdf[(tempdf.task == 'unstr') & (tempdf.reward == 0) & (tempdf.sess_bin>=4) & tempdf.animal.isin(sham_an)].groupby('animal').shift_t0.mean().mean()

yr_unstr_err = tempdf[(tempdf.task == 'unstr') & (tempdf.reward == 1) & (tempdf.sess_bin>=4) & tempdf.animal.isin(sham_an)].groupby('animal').shift_t0.mean().sem()
ynr_unstr_err = tempdf[(tempdf.task == 'unstr') & (tempdf.reward == 0) & (tempdf.sess_bin>=4)& tempdf.animal.isin(sham_an)].groupby('animal').shift_t0.mean().sem()

yr_str = tempdf[(tempdf.task == 'sham') & (tempdf.reward == 1) & tempdf.animal.isin(sham_an)].groupby('animal').shift_t0.mean().mean()
ynr_str = tempdf[(tempdf.task == 'sham') & (tempdf.reward == 0) & tempdf.animal.isin(sham_an)].groupby('animal').shift_t0.mean().mean()

yr_str_err = tempdf[(tempdf.task == 'sham') & (tempdf.reward == 1) & tempdf.animal.isin(sham_an)].groupby('animal').shift_t0.mean().sem()
ynr_str_err = tempdf[(tempdf.task == 'sham') & (tempdf.reward == 0) & tempdf.animal.isin(sham_an)].groupby('animal').shift_t0.mean().sem()

plt.figure(figsize=(3.5,4))
width = 0.4
plt.bar(0 - width/2, yr_unstr, color = 'red', yerr = yr_unstr_err, width = width)
plt.bar(0 + width/2, ynr_unstr, color = 'blue', yerr = yr_unstr_err, width = width)

plt.bar(1 - width/2, yr_str, color = 'red', yerr = yr_str_err, width = width)
plt.bar(1 + width/2, ynr_str, color = 'blue', yerr = ynr_str_err, width = width)
plt.ylim(0, 0.45)
plt.xticks([0, 1], ['Pre-lesion', 'Post-lesion'], fontsize = 'large', color = 'grey')
plt.ylabel('Switch rate', fontsize = 'x-large')
plt.yticks([0, 0.2, 0.4], ['0.0', 0.2, 0.4], fontsize = 'large', color = 'grey')
sns.despine()

# Statistical annotations
from scipy.stats import ttest_rel

# Rewarded pre vs post
rewarded_pre = tempdf[(tempdf.task == 'unstr') & (tempdf.reward == 1) & (tempdf.sess_bin>=4) & tempdf.animal.isin(sham_an)].groupby('animal').shift_t0.mean()
rewarded_post = tempdf[(tempdf.task == 'sham') & (tempdf.reward == 1) & tempdf.animal.isin(sham_an)].groupby('animal').shift_t0.mean()
t_stat, p_val = ttest_rel(rewarded_pre, rewarded_post)

# Unrewarded pre vs post
unrewarded_pre = tempdf[(tempdf.task == 'unstr') & (tempdf.reward == 0) & (tempdf.sess_bin>=4) & tempdf.animal.isin(sham_an)].groupby('animal').shift_t0.mean()
unrewarded_post = tempdf[(tempdf.task == 'sham') & (tempdf.reward == 0) & tempdf.animal.isin(sham_an)].groupby('animal').shift_t0.mean()
t_stat2, p_val2 = ttest_rel(unrewarded_pre, unrewarded_post)

# Add annotations
max_nr = max(ynr_unstr + ynr_unstr_err, ynr_str + ynr_str_err)-0.02

if p_val < 0.05:
    plt.plot([0 - width/2, 1 - width/2], [max_nr + 0.05, max_nr + 0.05], color='black', linewidth=1)
    plt.text(0.5, max_nr + 0.055, f'p={p_val:.3f}', ha='center', fontsize=10)
else:
    plt.plot([0 - width/2, 1 - width/2], [max_nr + 0.05, max_nr + 0.05], color='black', linewidth=1)
    plt.text(0.5, max_nr + 0.055, 'ns', ha='center', fontsize=10)

if p_val2 < 0.05:
    plt.plot([0 + width/2, 1 + width/2], [max_nr + 0.1, max_nr + 0.1], color='black', linewidth=1)
    plt.text(0.5, max_nr + 0.105, f'p={p_val2:.3f}', ha='center', fontsize=10)
else:
    plt.plot([0 + width/2, 1 + width/2], [max_nr + 0.1, max_nr + 0.1], color='black', linewidth=1)
    plt.text(0.5, max_nr + 0.105, 'ns', ha='center', fontsize=10)
plt.tight_layout()

# metric vs/ metric correlation

In [248]:
# plot perf index for each animal vs bias
trialsinsess = 100
tempdf = sessdf.groupby(['animal','session']).filter(lambda x: x.reward.size >= trialsinsess).groupby(['animal','session']).head(trialsinsess)

In [254]:
# calculate entropy per session
# tempdf[(tempdf.task == 'unstr')&(tempdf.sess_bin>=4)].groupby(['animal', 'session']).entropy.mean().unstack().mean(axis = 1)

# convert to numpy array
# g = tempdf.groupby(['animal', 'session']).cumcount()
# L = np.array(tempdf.set_index(['session',g])
#         .unstack(fill_value=0)
#         .stack().groupby(level=0)
#         .apply(lambda x: x.port.values.tolist())
#         .tolist())
# # mean
# entropy_trial = [entropy(calc_prob(L[:, col]), base = 2) for col in range(L.shape[1])]

y = tempdf[(tempdf.task == 'unstr')&(tempdf.sess_bin>=4)].groupby(['animal', 'session']).port.apply(lambda x: entropy(calc_prob(x[:10]), base = 2)).unstack().mean(axis = 1)

x = tempdf[(tempdf.task == 'unstr')&(tempdf.sess_bin>=4)].groupby(['animal', 'session']).rr.mean().unstack().mean(axis = 1)
# sns.regplot(x =x, y=y, ci = None)
# regress x against y and draw a line with coefficient and intercept from model
from sklearn.linear_model import LinearRegression
from scipy.stats import linregress
model = LinearRegression()
model.fit(x.values.reshape(-1, 1), y.values)
r_sq = model.score(x.values.reshape(-1, 1), y.values)
plt.figure()
plt.scatter(x, y, color = 'grey')
plt.plot(x, model.predict(x.values.reshape(-1, 1)), color = 'red')
slope, intercept, r_value, p_value, std_err = linregress(x,y)
print(p_value, slope, intercept)
print(model.coef_, model.intercept_)
plt.title(fr'$R^2$ = {round(r_sq, 4)}, p = {round(p_value, 4)}', fontsize = 'large')
plt.xlabel('Reward rate', fontsize = 'x-large')

plt.ylabel('Entropy', fontsize = 'x-large')
sns.despine()
plt.tight_layout()

# sns.regplot(x = x, y = y, ci = None)

0.018206987697983328 -1.7331756284868047 1.734430831227499
[-1.73317563] 1.734430831227499


In [ ]:
y = tempdf[(tempdf.task == 'unstr')&(tempdf.sess_bin>=4)].groupby(['animal', 'session']).shift_t0.mean().unstack().mean(axis = 1)
from sklearn.linear_model import LinearRegression
from scipy.stats import linregress
model = LinearRegression()
model.fit(x.values.reshape(-1, 1), y.values)
r_sq = model.score(x.values.reshape(-1, 1), y.values)
plt.figure()
plt.scatter(x, y, color = 'grey')
plt.plot(x, model.predict(x.values.reshape(-1, 1)), color = 'red')
slope, intercept, r_value, p_value, std_err = linregress(x,y)
print(p_value, slope, intercept)
print(model.coef_, model.intercept_)
plt.title(fr'$R^2$ = {round(r_sq, 4)}, p = {round(p_value, 4)}', fontsize = 'large')
plt.xlabel('Performance index', fontsize = 'x-large')
plt.ylabel('Average switch probability', fontsize = 'x-large')
sns.despine()
plt.tight_layout()

In [662]:
y = tempdf[(tempdf.task == 'unstr')&(tempdf.sess_bin>=4)&(tempdf.shift_t0 == 1)].groupby(['animal', 'session']).disp.apply(lambda x: np.mean(abs(x))).unstack().mean(axis = 1)
from sklearn.linear_model import LinearRegression
from scipy.stats import linregress
model = LinearRegression()
model.fit(x.values.reshape(-1, 1), y.values)
r_sq = model.score(x.values.reshape(-1, 1), y.values)
plt.figure()
plt.scatter(x, y, color = 'grey')
plt.plot(x, model.predict(x.values.reshape(-1, 1)), color = 'red')
slope, intercept, r_value, p_value, std_err = linregress(x,y)
print(p_value, slope, intercept)
print(model.coef_, model.intercept_)
plt.title(fr'$R^2$ = {round(r_sq, 4)}, p = {round(p_value, 4)}', fontsize = 'large')
plt.xlabel('Performance index', fontsize = 'x-large')
plt.ylabel('Average switch distance', fontsize = 'x-large')
sns.despine()
plt.tight_layout()

0.12515671710251056 -2.634573790691113 3.363466412685049
[-2.63457379] 3.363466412685049
